# 1. Хеширование. 
Хеширование — это процесс преобразования входных данных (обычно строки) в фиксированный набор байтов (хеш-значение) с помощью хеш-функции. Хеш-функции широко используются в различных областях, таких как хранение данных, проверка целостности, реализация таблиц хеш-таблиц и построение структур данных.

Основные характеристики хеш-функций:
1) Однонаправленность : Из хеш-значения невозможно восстановить исходные данные.
2) Быстрота вычисления : Хеш-функция должна быстро вычислять хеш-значение для любого входного данных.
3) Совпадение минимальное (коллизии) : Вероятность того, что два разных входных данных имеют одинаковое хеш-значение, должна быть минимальной.
4) Детерминированность : Для одинаковых входных данных хеш-функция всегда должна возвращать одинаковое хеш-значение.

# Алгоритм Рабина-Карпа.
Алгоритм Рабина-Карпа — это эффективный алгоритм поиска подстроки в строке. Он использует хеширование для быстрого сравнения подстрок.

Основные идеи алгоритма:
1) Хеширование подстроки и окна : Вычисляется хеш-значение искомой подстроки и каждого окна в строке одинаковой длины.
2) Сравнение хеш-значений : Если хеш-значения совпадают, выполняется дополнительное сравнение символов для подтверждения совпадения.
3) Смещение окна : Окно смещается на один символ вправо, и хеш-значение обновляется.
Преимущества алгоритма:
1) Эффективность : Средняя временная сложность — O(n + m), где n — длина строки, m — длина подстроки.
2) Простота реализации : Алгоритм относительно прост в реализации и понимании.

In [ ]:
def polhash(data, a, b):
    res = 0
    for x in data:
        res = (res * a + ord(x) - ord('a') + 1) % b
    return res

In [ ]:
def rabin_karp(A, B):
    # Если длина подстроки A больше строки B, то подстрока A не может быть в B
    if len(A) > len(B):
        return [-1]

    # Основание системы счисления для хеширования
    base = 91
    # Модуль для хеширования, большое простое число
    modulus = 1000000321

    # Функция для вычисления полиномиального хеша строки s длиной length
    def polhash(s, length):
        h = 0
        for i in range(length):
            # Вычисляем хеш, добавляя символ s[i] и учитывая основание и модуль
            h = (h * base + ord(s[i])) % modulus
        return h

    # Вычисляем хеш подстроки A
    hash_A = polhash(A, len(A))
    # Вычисляем хеш первого окна в строке B длиной len(A)
    hash_B = polhash(B, len(A))

    # Вычисляем значение base^(len(A)-1) % modulus для обновления хеша
    power = 1
    for _ in range(len(A) - 1):
        power = (power * base) % modulus

    # Список для хранения результатов (индексов начала подстроки A в B)
    res = []

    # Проходим по строке B и проверяем каждый окно длиной len(A)
    for i in range(len(B) - len(A) + 1):
        # Если хеши совпадают, проверяем символы в окне
        if hash_A == hash_B:
            # Если подстроки совпадают, добавляем индекс в результат
            if A == B[i:i + len(A)]:
                res.append(i)

        # Обновляем хеш для следующего окна, если не достигли конца строки B
        if i < len(B) - len(A):
            # Удаляем старый символ из хеша
            hash_B = (hash_B - ord(B[i]) * power) % modulus
            # Добавляем новый символ в хеш
            hash_B = (hash_B * base + ord(B[i + len(A)])) % modulus
            # Корректируем хеш, если он стал отрицательным
            hash_B = (hash_B + modulus) % modulus

    # Возвращаем результаты или [-1], если совпадений нет
    return res if res else [-1]

# Ввод подстроки A
A = input()
# Ввод строки B
B = input()
# Вывод результатов, разделённых пробелами
print(" ".join(map(str, rabin_karp(A, B))))

# 2. Открытая хеш и закрытая хеш-таблицы. Проблема удаления из закрытой хеш-таблицы. Перехеширование.

### Открытая и закрытая хеш-таблицы

**Хеш-таблица** — это структура данных, которая использует хеш-функцию для преобразования ключей в индексы массива, что позволяет эффективно хранить и извлекать данные. Хеш-таблицы могут быть реализованы двумя основными способами: открытой и закрытой адресацией.

#### Открытая хеш-таблица

В открытой хеш-таблице (или открытой адресации) все элементы хранятся в одном массиве, и при коллизии (когда два ключа хешируются в один и тот же индекс) используется метод разрешения коллизий. Основные методы разрешения коллизий в открытой хеш-таблице:

1. **Линейное пробирование**: Если возникает коллизия, мы просто проверяем следующий индекс в массиве, пока не найдем пустую ячейку.
2. **Квадратичное пробирование**: Вместо того чтобы проверять следующий индекс, мы проверяем индексы, увеличивая шаг в квадрате (например, 1, 4, 9 и т.д.).
3. **Двойное хеширование**: Используется вторая хеш-функция для определения шага при коллизии.

Преимущества открытой хеш-таблицы:
- Простота реализации.
- Эффективное использование памяти, так как все элементы хранятся в одном массиве.

Недостатки:
- При высокой нагрузке (большом количестве коллизий) производительность может значительно ухудшиться.

#### Закрытая хеш-таблица

В закрытой хеш-таблице (или закрытой адресации) элементы хранятся в отдельной структуре данных (например, связанных списках или деревьях) для каждого индекса массива. При коллизии элементы с одинаковым хеш-значением добавляются в эту структуру.

Преимущества закрытой хеш-таблицы:
- Более высокая производительность при большом количестве коллизий, так как элементы не перемещаются по массиву.
- Легче управлять удалением элементов, так как они могут быть просто удалены из связанного списка или дерева.

Недостатки:
- Использует больше памяти, так как для каждого индекса может потребоваться дополнительная структура данных.

### Проблема удаления из закрытой хеш-таблицы

Удаление элементов из закрытой хеш-таблицы может быть проблематичным, особенно если используется связанный список для разрешения коллизий. Основные проблемы, связанные с удалением:

1. **Потеря информации о коллизиях**: Если элемент удаляется, и он был единственным в своем списке, то это не создает проблем. Однако, если в списке есть другие элементы, то удаление может привести к тому, что некоторые из них станут недоступными, если не будет правильно обновлен указатель на следующий элемент.

2. **Необходимость поддержания структуры**: После удаления элемента необходимо убедиться, что структура данных (например, связанный список) остается корректной. Это может потребовать дополнительных операций, что увеличивает сложность.

3. **Проблемы с производительностью**: Если удаление происходит часто, это может привести к фрагментации памяти и ухудшению производительности, так как структура данных может стать неэффективной.

### Перехеширование

**Перехеширование** — это процесс, который используется для изменения размера хеш-таблицы и перераспределения элементов, когда таблица становится слишком заполненной или слишком пустой. Это может происходить по следующим причинам:

1. **Увеличение размера**: Когда количество элементов в хеш-таблице превышает определенный порог (например, 70% от общего размера), может потребоваться увеличение размера таблицы. Это делается для уменьшения вероятности коллизий и улучшения производительности.

2. **Уменьшение размера**: Если хеш-таблица становится слишком пустой (например, менее 30% заполненности), может быть целесообразно уменьшить ее размер, чтобы сэкономить память.

Процесс перехеширования включает следующие шаги:
1. Создание новой, более крупной или меньшей хеш-таблицы.
2. Пересчет хеш-значений для всех существующих элементов.
3. Вставка элементов в новую таблицу.

Перехеширование может быть затратным по времени, так как требует повторного вычисления хеш-значений и перемещения элементов. Однако оно необходимо для поддержания эффективности хеш-таблицы в долгосрочной перспективе.

In [ ]:
def polhash(data:str, a:int, b:int)->int:
    res = 0
    for x in data:
        res = (res * a + ord(x)) % b
    return res

def insert(table, key, value):
    hash = polhash(key, 91, 100)
    index = hash % 10
    for item in table[index]:
        if item[1] == key:
            item[2] = value  
            return
        
    table[index].append([hash, key, value])

def search(table, key:str):
    hash = polhash(key, 91, 100)
    index = hash % 10
    for i in table[index]:
            if i[0] == hash and i[1] == key:
                return i[2] 
    return 'KeyError'

def remove(table, key:str):
    hash = polhash(key, 91, 100)
    index = hash % 10
    for i in table[index]:
            if i[0] == hash and i[1] == key:
                table[index].remove(i)
                return i[2]
    return 'KeyError'

hash_table = [[] for _ in range(10)]
M = int(input())
for _ in range(M):
    key, value = map(str, input().split())
    insert(hash_table, key, value)

for i in range(len(hash_table)):
    if hash_table[i] == []:
        continue
    else:
        print(i)
        for j in range(len(hash_table[i])):
            print(*hash_table[i][j])


# 3. Словари и множества в Python.
Словари и множества — это две важные структуры данных в Python, которые позволяют эффективно хранить и обрабатывать данные.

### Словари (dict)

**Словарь** — это неупорядоченная коллекция пар "ключ-значение". Каждый ключ в словаре должен быть уникальным, и он используется для доступа к соответствующему значению. Словари реализованы с использованием хеш-таблиц, что обеспечивает быструю вставку, удаление и поиск элементов.

#### Основные свойства словарей:
- Неупорядоченные: до Python 3.6 порядок не гарантировался, начиная с Python 3.7 порядок сохраняется.
- Ключи должны быть неизменяемыми (например, строки, числа, кортежи).
- Значения могут быть любого типа и могут дублироваться.

### Множества (set)

**Множество** — это неупорядоченная коллекция уникальных элементов. Множества также реализованы с использованием хеш-таблиц, что обеспечивает быструю проверку на наличие элемента, добавление и удаление.

#### Основные свойства множеств:
- Неупорядоченные: элементы не имеют фиксированного порядка.
- Все элементы должны быть уникальными.
- Множества могут содержать только неизменяемые (хешируемые) типы данных.

### Заключение

Словари и множества в Python — это мощные структуры данных, которые позволяют эффективно хранить и обрабатывать данные. Словари удобны для хранения пар "ключ-значение", а множества — для работы с уникальными элементами и выполнения различных операций над ними. Оба типа данных обеспечивают быструю работу благодаря использованию хеш-таблиц.

In [ ]:
# Создание словаря
person = {
    "name": "Alice",
    "age": 30,
    "city": "New York"
}

# Доступ к элементам
print(person["name"])  # Вывод: Alice
print(person.get("age"))  # Вывод: 30

# Изменение значения
person["age"] = 31

# Добавление нового элемента
person["job"] = "Engineer"

# Удаление элемента
del person["city"]

# Перебор ключей и значений
for key, value in person.items():
    print(f"{key}: {value}")

# Создание множества
fruits = {"apple", "banana", "cherry"}

# Добавление элемента
fruits.add("orange")

# Удаление элемента
fruits.remove("banana")

# Проверка на наличие элемента
if "apple" in fruits:
    print("Apple is in the set")

# Перебор элементов
for fruit in fruits:
    print(fruit)

# 4. Списки: односвязный, двусвязный, кольцо. Время работы основных операций (добавление в начало/конец, удаление с начала/конца, обращение к произвольному элементу).

Списки — это структуры данных, которые позволяют хранить коллекции элементов. В зависимости от реализации, списки могут иметь разные свойства и временные характеристики для выполнения основных операций. Рассмотрим три типа списков: односвязный, двусвязный и кольцевой.

### 1. Односвязный список

**Односвязный список** — это структура данных, в которой каждый элемент (узел) содержит данные и ссылку на следующий узел. Последний узел указывает на `None`, что обозначает конец списка.

#### Время работы основных операций:
- **Добавление в начало**: O(1) — просто создаем новый узел и перенаправляем его на текущую голову списка.
- **Добавление в конец**: O(n) — необходимо пройти весь список, чтобы найти последний узел.
- **Удаление с начала**: O(1) — просто перенаправляем голову на следующий узел.
- **Удаление с конца**: O(n) — необходимо пройти весь список, чтобы найти предпоследний узел.
- **Обращение к произвольному элементу**: O(n) — необходимо пройти список до нужного элемента.

### 2. Двусвязный список

**Двусвязный список** — это структура данных, в которой каждый узел содержит данные, ссылку на следующий узел и ссылку на предыдущий узел. Это позволяет легко перемещаться в обоих направлениях.

#### Время работы основных операций:
- **Добавление в начало**: O(1) — создаем новый узел и перенаправляем ссылки.
- **Добавление в конец**: O(1) — если у нас есть ссылка на последний узел, мы можем добавить новый узел сразу.
- **Удаление с начала**: O(1) — просто перенаправляем голову на следующий узел.
- **Удаление с конца**: O(1) — если у нас есть ссылка на последний узел, мы можем удалить его сразу.
- **Обращение к произвольному элементу**: O(n) — необходимо пройти список до нужного элемента.

### 3. Кольцевой список

**Кольцевой список** — это структура данных, в которой последний узел указывает на первый узел, образуя кольцо. Кольцевые списки могут быть как односвязными, так и двусвязными.

#### Время работы основных операций:
- **Добавление в начало**: O(1) — создаем новый узел

In [ ]:
# Определение класса Node (Узел связного списка)
class Node:
    # Конструктор класса Node
    def __init__(self, data, next=None):
        self.data = data  # Храним данные в узле
        self.next = next    # Ссылка на следующий узел (по умолчанию None)

    # Метод для вставки нового узла после текущего
    def insert(self, data):
        n = Node(data, self.next)  # Создаем новый узел, который ссылается на следующий узел текущего
        self.next = n             # Теперь текущий узел ссылается на новый

    # Метод для удаления следующего узла
    def remove(self):
        r = next.data        # ОШИБКА: должно быть self.next.data (получаем данные следующего узла)
        self.next = self.next.next  # "Пропускаем" следующий узел, ссылаясь через один
        return r             # Возвращаем данные удаленного узла
    

# Класс LinkedList (Связный список)
class LinkedList:
    # Конструктор класса LinkedList
    def __init__(self):
        self.head = None  # Начало списка
        self.tail = None  # Конец списка (не используется в некоторых методах)

    # Добавление элемента в начало списка
    def add(self, data):
        n = Node(data, self.head)  # Создаем новый узел, который ссылается на текущий head
        if self.head is None:      # Если список пуст
            self.head = self.tail = n  # Новый узел становится и head, и tail
        else:
            self.head = n          # Новый узел становится новым head

    # ОШИБКА: метод append() реализован неверно - он делает то же самое, что и add()
    # Должен добавлять элемент в конец списка, а не в начало
    def append(self, data):
        n = Node(data, self.head)  # Здесь ошибка - новый узел ссылается на head
        if self.head is None:
            self.head = self.tail = n
        else:
            self.head = n          # Опять добавляем в начало, а не в конец

    # Удаление и возврат первого элемента списка
    def pop(self):
        if self.head is None:      # Если список пуст
            raise ValueError('pop from empty Linked List')
        res = self.head.data       # Сохраняем данные первого узла
        self.head = self.head.next  # Перемещаем head на следующий узел
        if self.head is None:      # Если список стал пустым
            self.tail = None       # Обнуляем tail
        return res                 # Возвращаем данные удаленного узла
    
    # Проверка, пуст ли список
    def is_empty(self):
        return self.head is None  # True, если head равен None
    

if __name__ == "__main__":
    from random import randint
    l = LinkedList()  # Создаем пустой связный список
    
    # Добавляем 10 случайных чисел в список
    for _ in range(10):
        x = randint(10, 99)  # Генерируем случайное число
        print(x, end=' ')    # Печатаем его
        l.add(x)             # Добавляем в начало списка

    # Извлекаем и печатаем все элементы, пока список не станет пустым
    while not l.is_empty():
        print(l.pop(), end=' ')  # Печатаем элементы в порядке извлечения

    print()  # Печатаем пустую строку для красоты

# 5. Двоичное дерево поиска.
# **Двоичное дерево поиска (Binary Search Tree, BST)**

## **Что это?**
**Двоичное дерево поиска (BST)** — это структура данных, в которой каждый узел имеет не более двух дочерних узлов (левый и правый). Для любого узла:
- Все элементы в **левом поддереве** меньше его значения.
- Все элементы в **правом поддереве** больше его значения.

### **Свойства BST:**
1. **Уникальность ключей** (обычно, но иногда допускаются дубликаты).
2. **Быстрый поиск** (в среднем за `O(log n)`, но в худшем случае `O(n)` при несбалансированном дереве).
3. **Упорядоченность** (обход **in-order** даёт отсортированную последовательность).

## **Вывод**
- **BST** позволяет эффективно выполнять вставку, удаление и поиск (`O(log n)` в среднем).
- **Недостаток**: если дерево становится несбалансированным (например, при вставке отсортированных данных), сложность операций ухудшается до `O(n)`.
- **Альтернативы**: **AVL-деревья** и **Красно-черные деревья** (автоматически балансируются).

In [ ]:
class Node:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None

class BinarySearchTree:
    def __init__(self):
        self.root = None  # Корень дерева

    def insert(self, key):
        """Вставка нового узла."""
        if self.root is None:
            self.root = Node(key)
        else:
            self._insert_recursive(self.root, key)

    def _insert_recursive(self, node, key):
        """Рекурсивная вставка."""
        if key < node.key:
            if node.left is None:
                node.left = Node(key)
            else:
                self._insert_recursive(node.left, key)
        elif key > node.key:
            if node.right is None:
                node.right = Node(key)
            else:
                self._insert_recursive(node.right, key)
        # Если ключ уже есть, ничего не делаем (можно обработать иначе)

    def search(self, key):
        """Поиск узла по ключу."""
        return self._search_recursive(self.root, key)

    def _search_recursive(self, node, key):
        """Рекурсивный поиск."""
        if node is None or node.key == key:
            return node
        if key < node.key:
            return self._search_recursive(node.left, key)
        return self._search_recursive(node.right, key)

    def delete(self, key):
        """Удаление узла."""
        self.root = self._delete_recursive(self.root, key)

    def _delete_recursive(self, node, key):
        """Рекурсивное удаление."""
        if node is None:
            return node
        if key < node.key:
            node.left = self._delete_recursive(node.left, key)
        elif key > node.key:
            node.right = self._delete_recursive(node.right, key)
        else:
            # Узел с одним или без детей
            if node.left is None:
                return node.right
            elif node.right is None:
                return node.left
            # Узел с двумя детьми: находим минимальный в правом поддереве
            temp = self._find_min(node.right)
            node.key = temp.key
            node.right = self._delete_recursive(node.right, temp.key)
        return node

    def _find_min(self, node):
        """Находит минимальный узел в поддереве."""
        current = node
        while current.left is not None:
            current = current.left
        return current

    def inorder_traversal(self):
        """Обход дерева в порядке in-order (сортировка по возрастанию)."""
        result = []
        self._inorder_recursive(self.root, result)
        return result

    def _inorder_recursive(self, node, result):
        if node:
            self._inorder_recursive(node.left, result)
            result.append(node.key)
            self._inorder_recursive(node.right, result)

# 6. Куча. Сортировка кучей.
### Куча (Heap)

**Куча** — это специальная структура данных, которая представляет собой полное двоичное дерево, удовлетворяющее свойству кучи. Существует два основных типа кучи:

1. **Макс-куча (Max-Heap)**: В этой структуре для каждого узла выполняется условие, что значение узла больше или равно значениям его дочерних узлов. Таким образом, корень дерева содержит максимальное значение.

2. **Мин-куча (Min-Heap)**: В этой структуре для каждого узла выполняется условие, что значение узла меньше или равно значениям его дочерних узлов. Корень дерева содержит минимальное значение.

### Основные операции с кучей

Куча поддерживает несколько основных операций:

1. **Вставка (Insert)**: Добавление нового элемента в кучу. Элемент добавляется в конец дерева, а затем происходит "всплытие" (sift-up) для восстановления свойства кучи.

2. **Удаление корня (Extract)**: Удаление корневого элемента (максимального или минимального). Корень заменяется последним элементом в куче, и затем происходит "погружение" (sift-down) для восстановления свойства кучи.

3. **Построение кучи (Heapify)**: Преобразование массива в кучу. Это можно сделать за O(n) времени.

4. **Просмотр корня (Peek)**: Получение значения корня без его удаления.

### Время работы операций

- **Вставка**: O(log n)
- **Удаление корня**: O(log n)
- **Построение кучи**: O(n)
- **Просмотр корня**: O(1)

### Сортировка кучей (Heap Sort)

**Сортировка кучей** — это алгоритм сортировки, который использует структуру данных "куча". Он работает следующим образом:

1. **Построение кучи**: Преобразуем массив в кучу (макс-кучу для сортировки по возрастанию).

2. **Удаление корня**: Извлекаем корень (максимальный элемент) и помещаем его в конец массива. Затем уменьшаем размер кучи и восстанавливаем ее свойства.

3. **Повторение**: Повторяем шаг 2, пока не отсортируем весь массив.

### Принципы работы сортировки кучей

1. **Эффективность**: Сортировка кучей имеет временную сложность O(n log n) в худшем, среднем и лучшем случаях, что делает ее эффективной для сортировки больших массивов.

2. **Не требует дополнительной памяти**: Сортировка кучей выполняется на месте, что означает, что она не требует дополнительной памяти для хранения временных массивов, как это делает, например, сортировка слиянием.

3. **Стабильность**: Сорт

In [ ]:
class Heap:
    def __init__(self):
        self.__h = []

    def push(self, el):
        self.__h.append(el)
        self.__shift_up(len(self.__h) - 1)

    def __shift_up(self, n):
        if n == 0: return
        p = (n - 1) // 2
        if self.__h[p] > self.__h[n]:
            self.__h[p], self.__h[n] = self.__h[n], self.__h[p]
            self.__shift_up(p)

    def pop(self):
        res = self.__h[0]
        self.__h[0] = self.__h[-1]
        self.__h.pop()
        if len(self.__h):
            self.__shift_down(0)
        return res
    
    def __shift_down(self, i):
        maxN = len(self.__h) if maxN is None else maxN
        m = self.__h[i]
        if 2 * i + 1 < maxN: m = min(m, self.__h[2 * i + 1])
        if 2 * i + 2 < maxN: m = min(m, self.__h[2 * i + 2])
        if self.__h[i] == m: return
        if self.__h[2 * i + 1] == m:
            self.__h[i], self.__h[2*i+1] = self.__h[2*i+1], self.__h[i]
            self.__shift_down(2 * i + 1, maxN)
            return
        if self.__h[2 * i + 2] == m:
            self.__h[i], self.__h[2*i+2] = self.__h[2*i+2], self.__h[i]
            self.__shift_down(2 * i + 2, maxN)
    
    @staticmethod
    def heapify(a):
        h = Heap()
        h.__h = a[:]
        for i in range((len(a)-1)//2, -1, -1):
            h.__shift_down(i)
        return h
    
    @staticmethod
    def sort(a):
        h = Heap()
        h.__h = a 
        for i in range((len(a)-1)//2, -1, -1):
            h.__shift_down(i)
            
        for i in range((len(a)-1), 0, -1):
            h.__h[0], h.__h[i] = h.__h[i], h.__h[0]
            h.__shift_down(0, i)


if __name__ ==  "__main__":
    from random import randint

    h = Heap()
    for _ in range(10):
        x = randint(10, 99)
        print(x ,end=' ')
        h.push(x)

    print()
    print(h._Heap__h)
    for _ in range(10):
        print(h.pop(), end=' ')

    print()

    a = [randint(10,99) for i in range(10)]
    b = Heap.heapify(a)
    print(a)
    print(b._Heap__h)
    Heap.sort(a)
    print(a)

# 7. Определение графа. Степень вершины, петли, кратные рёбра. Цепи, пути и циклы.
# **Графы: основные определения и понятия**

## **1. Что такое граф?**
**Граф** — это математическая структура, состоящая из:
- **Вершин (узлов)** (обозначаются обычно `V` или `U, V, ...`)
- **Рёбер (дуг)** (обозначаются `E`), которые соединяют вершины.

### **Виды графов:**
- **Неориентированный граф** — рёбра не имеют направления (`(A, B)` = `(B, A)`).
- **Ориентированный граф (орграф)** — рёбра имеют направление (`A → B` ≠ `B → A`).
- **Взвешенный граф** — каждому ребру присвоен вес (число).
- **Не взвешенный граф** — рёбра не имеют весов.

---

## **2. Степень вершины**
- **В неориентированном графе**:
  - **Степень вершины** (`deg(v)`) — количество рёбер, инцидентных вершине.
  - **Петля** (ребро `(v, v)`) увеличивает степень вершины на **2**.
  
  **Пример**:  
  ![Неориентированный граф](https://i.imgur.com/xyz123.png)  
  `deg(A) = 3`, `deg(B) = 2`, `deg(C) = 1`.

- **В ориентированном графе**:
  - **Полустепень исхода (`out-degree`)** — число выходящих рёбер.
  - **Полустепень захода (`in-degree`)** — число входящих рёбер.
  
  **Пример**:  
  ![Ориентированный граф](https://i.imgur.com/abc456.png)  
  `out-degree(A) = 2`, `in-degree(A) = 1`.

---

## **3. Петли и кратные рёбра**
- **Петля** — ребро, соединяющее вершину саму с собой (`(A, A)`).  
  ![Петля](https://i.imgur.com/def789.png)  

- **Кратные рёбра** — несколько рёбер между одной парой вершин.  
  ![Кратные рёбра](https://i.imgur.com/ghi012.png)  

- **Простой граф** — граф **без петель и кратных рёбер**.
- **Мультиграф** — граф **с кратными рёбрами и/или петлями**.

---

## **4. Цепи, пути и циклы**
### **В неориентированном графе:**
- **Цепь** — последовательность рёбер, соединяющих вершины **без повторений рёбер**.  
  (Могут повторяться вершины, кроме замкнутых случаев).  
  **Пример**: `A — B — C — A — D` (цепь).  

- **Простая цепь** — цепь **без повторений вершин**.  
  **Пример**: `A — B — C — D` (простая цепь).  

- **Цикл** — замкнутая цепь (`начало = конец`).  
  **Пример**: `A — B — C — A` (цикл длины 3).  

### **В ориентированном графе:**
- **Путь** — последовательность **ориентированных рёбер** `v₁ → v₂ → ... → vₙ`.  
  **Пример**: `A → B → C → D` (путь).  

- **Контур** — замкнутый путь (`начало = конец`).  
  **Пример**: `A → B → C → A` (контур).  

---

## **5. Пример графа и его свойств**
```
        A
      / | \
     B  C  D
      \ | /
        E
```
- **Степени вершин**:  
  `deg(A) = 3`, `deg(B) = 2`, `deg(C) = 2`, `deg(D) = 2`, `deg(E) = 3`.  

- **Пример цепи**: `A → B → E → C → A` (не простая, т.к. `A` повторяется).  
- **Пример простой цепи**: `A → D → E → B`.  
- **Пример цикла**: `A → B → E → A`.  

# 8. Сильная и слабая связность графа. Компоненты связности.

## **1. Связность в графах**
### **Неориентированные графы**
- **Связный граф** — если между **любыми двумя вершинами** существует путь.
- **Компонента связности** — максимальный связный подграф (все вершины достижимы друг из друга).

**Пример**:  
```
A — B    C — D  
 \ /     |  
  E     F — G
```
- **Компоненты связности**:  
  - `{A, B, E}`  
  - `{C, D, F, G}`  

---

### **Ориентированные графы (орграфы)**
Здесь понятие связности сложнее из-за направленности рёбер.

#### **1. Сильная связность**
- Орграф **сильно связный**, если для **любых двух вершин `u` и `v`** существует:  
  - путь из `u` в `v`,  
  - и путь из `v` в `u`.  
- **Компонента сильной связности (КСС)** — максимальный сильно связный подграф.

**Пример**:  
```
A → B → C  
↑   ↓   ↓  
D ← E ← F
```
- **КСС**:  
  - `{A, B, E, D}` (можно добраться из любой вершины в любую).  
  - `{C}` и `{F}` — отдельные КСС, так как из `C` нельзя вернуться.  

#### **2. Слабая связность**
- Орграф **слабо связный**, если при **игнорировании направлений рёбер** он становится связным неориентированным графом.
- **Компонента слабой связности** — аналогично компоненте в неориентированном графе.

**Пример**:  
```
A → B ← C  
 \   /  
   D
```
- **Слабо связный** (если сделать рёбра неориентированными, все вершины связаны).  
- **Не сильно связный** (из `C` нельзя попасть в `A`).  

---

## **2. Алгоритмы поиска компонент связности**
### **Для неориентированных графов**  
Используется **DFS (Depth-First Search)** или **BFS (Breadth-First Search)**.  

**Алгоритм**:  
1. Выбрать непосещённую вершину.  
2. Запустить DFS/BFS, пометив все достижимые вершины.  
3. Эти вершины образуют компоненту связности.  
4. Повторять, пока есть непосещённые вершины.  

### **Для ориентированных графов (поиск КСС)**  
Используется **алгоритм Косарайю** или **Тарьяна**.  

**Алгоритм Косарайю**:  
1. **Первый обход DFS** — запоминаем порядок выхода.  
2. **Транспонируем граф** (разворачиваем рёбра).  
3. **Второй обход DFS** в порядке убывания времени выхода.  

---

## **Вывод**
| Понятие                | Неориентированный граф | Ориентированный граф |
|------------------------|------------------------|-----------------------|
| **Связность**          | Существует путь между любыми вершинами | Сильная (двусторонние пути) / слабая (без учёта направлений) |
| **Компоненты**         | Компоненты связности   | Компоненты сильной связности (КСС) |
| **Алгоритмы**          | DFS/BFS                | Косарайю, Тарьян |

In [ ]:
from collections import defaultdict
def find_components(graph):
    visited = set()
    components = []
    
    for node in graph:
        if node not in visited:
            stack = [node]
            visited.add(node)
            component = []
            
            while stack:
                current = stack.pop()
                component.append(current)
                
                for neighbor in graph[current]:
                    if neighbor not in visited:
                        visited.add(neighbor)
                        stack.append(neighbor)
            
            components.append(component)
    
    return components

# Пример графа (неориентированный)
graph = {
    'A': ['B', 'E'],
    'B': ['A', 'E'],
    'E': ['A', 'B'],
    'C': ['D', 'F'],
    'D': ['C', 'G'],
    'F': ['C', 'G'],
    'G': ['D', 'F']
}

print(find_components(graph)) 
# Вывод: [['A', 'E', 'B'], ['C', 'F', 'G', 'D']]

def kosaraju(graph):
    visited = set()
    order = []
    
    # Первый DFS для определения порядка
    def dfs(node):
        stack = [(node, False)]
        
        while stack:
            current, processed = stack.pop()
            if processed:
                order.append(current)
                continue
            if current in visited:
                continue
            visited.add(current)
            stack.append((current, True))
            for neighbor in graph.get(current, []):
                if neighbor not in visited:
                    stack.append((neighbor, False))
    
    for node in graph:
        if node not in visited:
            dfs(node)
    
    # Транспонируем граф
    reversed_graph = defaultdict(list)
    for u in graph:
        for v in graph[u]:
            reversed_graph[v].append(u)
    
    # Второй DFS в обратном порядке
    visited = set()
    components = []
    
    while order:
        node = order.pop()
        if node not in visited:
            stack = [node]
            visited.add(node)
            component = []
            
            while stack:
                current = stack.pop()
                component.append(current)
                for neighbor in reversed_graph.get(current, []):
                    if neighbor not in visited:
                        visited.add(neighbor)
                        stack.append(neighbor)
            
            components.append(component)
    
    return components

# Пример графа (ориентированный)
graph = {
    'A': ['B'],
    'B': ['C', 'E'],
    'C': ['A'],
    'D': ['B'],
    'E': ['D'],
    'F': ['E', 'G'],
    'G': ['F']
}

print(kosaraju(graph))
# Вывод: [['A', 'C', 'B', 'E', 'D'], ['F', 'G']]

# 9. Способы представления графа в памяти.

In [ ]:
# матрица смежности 
def read_matrix():
    N, M = map(int, input().split())
    G = [[0] * N for _ in range(N)]
    name = []
    for _ in range(M):
        v, u = input().split()
        if v not in name: name.append(v)
        if u not in name: name.append(u)
        v = name.index(v)
        u = name.index(u)
        G[u][v] = 1 # убрать, если граф оритентированный
        G[v][u] = 1
    return G, name

# список смежности
def read_list(): 
    N, M = map(int, input().split())
    G = [[] * N for _ in range(N)]
    name = []
    for _ in range(M):
        v, u = input().split()
        if v not in name: name.append(v)
        if u not in name: name.append(u)
        v = name.index(v)
        u = name.index(u)
        G[u].append(v) # убрать, если граф ориентированный
        G[v].append(u) 
    return G, name

# медленный (не для кр)
def read_graph(orient = False, weight = False):
    G = {}
    N, M = map(int, input().split())
    for _ in range(M):
        if not weight:
            u, v = input().split()
            if u not in G: G[u] = []
            if v not in G: G[v] = []
            G[u].append(v)
            if not orient: G[u].append(u)
        else:
            u, v, w = input().split()
            w = float(w)
            if u not in G: G[u] = {}
            if v not in G: G[v] = {}
            G[u][v] = w
            if not orient: G[u][v] = w
    return G

# компактный способ хранения графа
# реализация без петель 
def compact(G:list[list[int]]):
    offset = [0]
    edges = []
    for lst in G:
        for ends in lst:
            edges.append(ends)
        offset.append(len(edges))
    return offset, edges

if __name__ == "__main__":
    
    #1
    G, name = read_matrix()
    print(" ", *name)
    for i in range(len(name)):
        print(name[i], *G[i], sum(G[i]))

    #2
    G, name = read_list()
    for i in range(len(name)):
        print(name[i], ':', *map(lambda x:name[x], G[i]), len(G[i]))

    #3
    G = read_graph()
    for v in G:
        print(v, ":", *G[v], len(G[v]))


# 10. Выделение компоненты связности обходом в глубину.
# **Выделение компоненты связности обходом в глубину (DFS)**

## **1. Основная идея**
**Обход в глубину (DFS, Depth-First Search)** — это алгоритм, который исследует граф, двигаясь **как можно глубже** по одной ветви перед возвратом (backtracking).  

Для **выделения компоненты связности**:
1. Запускаем DFS из **произвольной непосещённой вершины**.
2. Все вершины, достижимые из неё, образуют **одну компоненту связности**.
3. Повторяем для оставшихся непосещённых вершин.

## **2. Алгоритм**

### **Для ориентированного графа (КСС)**
Используется **алгоритм Косарайю** (два прохода DFS):
1. Первый DFS — определяем порядок выхода.
2. Транспонируем граф.
3. Второй DFS — выделяем КСС.


## **3. Визуализация**
### **Неориентированный граф**
```
A — B — D    E — F
 \ /
  C
```
- **Компоненты**: `{A, B, C, D}` и `{E, F}`.

### **Ориентированный граф**
```
1 → 2 → 3 → 1
4 ↔ 5 → 6 → 3
```
- **КСС**:  
  - `{1, 2, 3}` (цикл),  
  - `{4, 5}` (взаимная связь),  
  - `{6}` (достижима, но не возвращается).


## **6. Сложность**
- **Время**: `O(V + E)` (каждый узел и ребро посещаются один раз).
- **Память**: `O(V)` (хранение посещённых вершин).

---

**Итог**: DFS — эффективный способ выделения компонент связности в графах. Для орграфов требуется модификация (Косарайю).

In [ ]:
#Для неориентированного графа
def dfs(node, graph, visited, component):
    visited.add(node)
    component.append(node)
    for neighbor in graph[node]:
        if neighbor not in visited:
            dfs(neighbor, graph, visited, component)

def find_connected_components(graph):
    visited = set()
    components = []
    for node in graph:
        if node not in visited:
            component = []
            dfs(node, graph, visited, component)
            components.append(component)
    return components

# для ориентированного 
def kosaraju(graph):
    visited = set()
    order = []

    def dfs_pass1(node):
        stack = [(node, False)]
        while stack:
            current, processed = stack.pop()
            if processed:
                order.append(current)
                continue
            if current in visited:
                continue
            visited.add(current)
            stack.append((current, True))
            for neighbor in graph.get(current, []):
                if neighbor not in visited:
                    stack.append((neighbor, False))

    # Первый проход DFS
    for node in graph:
        if node not in visited:
            dfs_pass1(node)

    # Транспонируем граф
    reversed_graph = {node: [] for node in graph}
    for u in graph:
        for v in graph[u]:
            reversed_graph[v].append(u)

    # Второй проход DFS
    visited = set()
    components = []
    while order:
        node = order.pop()
        if node not in visited:
            stack = [node]
            visited.add(node)
            component = []
            while stack:
                current = stack.pop()
                component.append(current)
                for neighbor in reversed_graph.get(current, []):
                    if neighbor not in visited:
                        visited.add(neighbor)
                        stack.append(neighbor)
            components.append(component)
    return components

# 11. Проверка двудольности графа.
# **Проверка двудольности графа**

## **1. Что такое двудольный граф?**
**Двудольный граф (Bipartite Graph)** — это граф, вершины которого можно разделить на **два непересекающихся множества** `U` и `V` так, что:
- Каждое ребро соединяет вершину из `U` с вершиной из `V`.
- **Нет рёбер** между вершинами внутри одного множества.

**Примеры двудольных графов**:
- Граф, где `U` — мужчины, `V` — женщины, а рёбра — брачные связи.
- Шахматная доска (вершины — клетки, рёбра — ходы коня).

---

## **2. Критерии двудольности**
Граф **двудольный** тогда и только тогда, когда он:
- **Не содержит циклов нечётной длины** (т.е. все циклы чётные).
- **Может быть раскрашен в 2 цвета** (например, красный и синий) так, что соседние вершины имеют разные цвета.

---

## **3. Алгоритм проверки двудольности**
Используется **модифицированный BFS или DFS** с раскраской вершин.

### **Алгоритм (BFS + раскраска)**:
1. Выбрать стартовую вершину, покрасить её в цвет `0`.
2. Для каждой вершины `u`:
   - Все её соседи `v` должны быть покрашены в цвет `1 - color[u]`.
   - Если сосед уже раскрашен и его цвет совпадает с `u` → граф **не двудольный**.
3. Если все вершины раскрашены без конфликтов → граф **двудольный**.


## **4. Визуализация**
### **Двудольный граф**:
```
0 (красный) —— 1 (синий)
 |               |
3 (синий) —— 2 (красный)
```
- **Множества**: `U = {0, 2}`, `V = {1, 3}`.

### **Недвудольный граф**:
```
0 (красный) —— 1 (синий)
 | \         /
 |  2 (красный)
 | /         \
3 (синий) —— 4 (???)  # Конфликт: 4 нельзя покрасить
```
- **Цикл нечётной длины** `0 → 1 → 2 → 0`.


## **6. Сложность**
- **Время**: `O(V + E)` (как BFS/DFS).
- **Память**: `O(V)` (хранение цветов).

**Итог**:  
Двудольность проверяется **раскраской в 2 цвета** (BFS/DFS). Если раскраска возможна без конфликтов — граф двудольный. Иначе — нет.

In [ ]:
from collections import deque

def is_bipartite(graph):
    if not graph:
        return True
    
    color = {}  # Словарь для хранения цветов (0 и 1)
    
    for node in graph:  # Проверяем все компоненты связности
        if node not in color:
            queue = deque([node])
            color[node] = 0
            
            while queue:
                u = queue.popleft()
                for v in graph[u]:
                    if v not in color:
                        color[v] = 1 - color[u]  # Инвертируем цвет
                        queue.append(v)
                    elif color[v] == color[u]:
                        return False
    return True

graph = {
    0: [1, 3],
    1: [0, 2],
    2: [1, 3],
    3: [0, 2]
}
print(is_bipartite(graph))  # True (можно разделить на {0, 2} и {1, 3})

graph = {
    0: [1, 2, 3],
    1: [0, 2],
    2: [0, 1, 3],
    3: [0, 2]
}
print(is_bipartite(graph))  # False (нельзя раскрасить без конфликтов)

def is_bipartite_dfs(graph):
    color = {}
    for node in graph:
        if node not in color:
            stack = [(node, 0)]
            color[node] = 0
            while stack:
                u, c = stack.pop()
                for v in graph[u]:
                    if v not in color:
                        color[v] = 1 - c
                        stack.append((v, color[v]))
                    elif color[v] == c:
                        return False
    return True


# 12. Проверка графа на ацикличность или нахождение цикла обходом в глубину.
# **Проверка графа на ацикличность и нахождение цикла с помощью DFS**

## **1. Основные понятия**
- **Ациклический граф** — граф, не содержащий циклов.
- **Цикл** — путь, у которого начальная и конечная вершины совпадают.

## **2. Алгоритм проверки на ацикличность с помощью DFS**
При обходе в глубину можно обнаружить цикл, если мы встречаем вершину, которая уже посещена и не является родительской.

### **Алгоритм**:
1. Запускаем DFS из каждой непосещённой вершины.
2. Для каждой вершины храним её **родителя** (предыдущую вершину в обходе).
3. Если мы встречаем уже посещённую вершину, которая **не является родителем** текущей, значит, найден цикл.

### **Код на Python (для неориентированного графа)**:
```python
def has_cycle_dfs(graph):
    visited = set()
    
    def dfs(node, parent):
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                if dfs(neighbor, node):
                    return True
            elif neighbor != parent:
                return True
        return False
    
    for node in graph:
        if node not in visited:
            if dfs(node, None):
                return True
    return False
```

### **Пример работы**:
```python
graph = {
    0: [1, 2],
    1: [0, 2],
    2: [0, 1, 3],
    3: [2]
}
print(has_cycle_dfs(graph))  # True (цикл 0-1-2-0)

graph = {
    0: [1],
    1: [0, 2],
    2: [1, 3],
    3: [2]
}
print(has_cycle_dfs(graph))  # False (нет циклов)
```

## **3. Для ориентированного графа**
В ориентированном графе нужно учитывать направление рёбер. Используется **дополнительный массив "в процессе"** (`in_stack`), чтобы отслеживать вершины в текущем пути DFS.

### **Код на Python (для ориентированного графа)**:
```python
def has_cycle_directed_dfs(graph):
    visited = set()
    in_stack = set()
    
    def dfs(node):
        visited.add(node)
        in_stack.add(node)
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                if dfs(neighbor):
                    return True
            elif neighbor in in_stack:
                return True
        in_stack.remove(node)
        return False
    
    for node in graph:
        if node not in visited:
            if dfs(node):
                return True
    return False
```

### **Пример работы**:
```python
graph = {
    0: [1],
    1: [2],
    2: [0]
}
print(has_cycle_directed_dfs(graph))  # True (цикл 0 → 1 → 2 → 0)

graph = {
    0: [1],
    1: [2],
    2: []
}
print(has_cycle_directed_dfs(graph))  # False (нет циклов)
```

## **4. Нахождение цикла**
Если нужно не только проверить наличие цикла, но и вывести его, можно модифицировать алгоритм.

### **Код для неориентированного графа**:
```python
def find_cycle_dfs(graph):
    visited = {}
    parent = {}
    cycle = []
    
    def dfs(node, par):
        visited[node] = True
        parent[node] = par
        for neighbor in graph[node]:
            if neighbor not in visited:
                if dfs(neighbor, node):
                    return True
            elif neighbor != par:
                # Найден цикл, восстанавливаем его
                current = node
                while current != neighbor:
                    cycle.append(current)
                    current = parent[current]
                cycle.append(neighbor)
                cycle.append(node)  # Замыкаем цикл
                return True
        return False
    
    for node in graph:
        if node not in visited:
            if dfs(node, None):
                return cycle[::-1]  # Разворачиваем для правильного порядка
    return None
```

### **Пример**:
```python
graph = {
    0: [1, 2],
    1: [0, 2],
    2: [0, 1, 3],
    3: [2]
}
print(find_cycle_dfs(graph))  # [0, 1, 2, 0]
```

## **5. Сложность**
- **Время**: `O(V + E)` (как у обычного DFS).
- **Память**: `O(V)` (хранение посещённых вершин и родителей).

## **6. Вывод**
- **Неориентированный граф**:  
  - Цикл есть, если при DFS встречается уже посещённая вершина, не являющаяся родителем.
- **Ориентированный граф**:  
  - Цикл есть, если при DFS встречается вершина, которая находится в текущем пути обхода (`in_stack`).
- **Нахождение цикла**:  
  - Можно восстановить, сохраняя родителей вершин.

In [ ]:
def is_acyclic(N, edges):
    # Создаем граф в виде списка смежности
    graph = {i: [] for i in range(N)}
    for u, v in edges:
        graph[u].append(v)
    
    # Массив для отслеживания посещенных вершин
    visited = [False] * N
    # Рекурсивный стек для отслеживания текущего пути
    rec_stack = [False] * N
    # Массив для хранения текущего пути
    path = []
    # Список для хранения вершин цикла
    cycle_vertices = []

    def dfs(v):
        # Помечаем текущую вершину как посещенную
        visited[v] = True
        # Помечаем текущую вершину как часть текущего пути
        rec_stack[v] = True
        # Добавляем текущую вершину в текущий путь
        path.append(v)
        
        # Проходим по всем соседним вершинам
        for neighbor in graph[v]:
            if rec_stack[neighbor]:
                # Обнаружен цикл
                # Найдем вершины цикла
                cycle_start = path.index(neighbor)
                cycle_vertices.extend(path[cycle_start:])
                return False
            if not visited[neighbor]:
                # Рекурсивно запускаем DFS для соседней вершины
                if not dfs(neighbor):
                    return False
        
        # Помечаем текущую вершину как не часть текущего пути
        rec_stack[v] = False
        # Удаляем текущую вершину из текущего пути
        path.pop()
        return True
    
    # Проверяем каждую вершину
    for i in range(N):
        if not visited[i]:
            if not dfs(i):
                # Выводим вершины цикла
                print("NO")
                print("Цикл:", " -> ".join(map(str, cycle_vertices)))
                return
    
    # Если цикл не обнаружен
    print("YES")

# Ввод данных
N = int(input("Введите количество вершин: "))
M = int(input("Введите количество ребер: "))

edges = []
for _ in range(M):
    u, v = map(int, input().split())
    edges.append((u, v))

# Проверка ацикличности
is_acyclic(N, edges)

from collections import deque

# Считываем входные данные
n, m = map(int, input().split())  # Число вершин и рёбер
graph = [[] for _ in range(n)]    # Создаём список смежности

# Считываем рёбра и строим граф
for _ in range(m):
    u, v = map(int, input().split())
    graph[u].append(v)  # Ребро идёт из u в v (граф ориентированный)

# Функция для поиска минимального цикла с использованием BFS
def find_shortest_cycle():
    min_cycle_length = float('inf')  # Минимальная длина цикла
    min_cycle_path = []              # Путь минимального цикла

    # Запускаем BFS из каждой вершины
    for start in range(n):
        # Массивы для хранения расстояний и предков
        distance = [-1] * n  # Расстояния до вершин
        parent = [-1] * n    # Предки для восстановления пути
        queue = deque([start])
        distance[start] = 0

        while queue:
            current = queue.popleft()

            # Обходим соседей текущей вершины
            for neighbor in graph[current]:
                if distance[neighbor] == -1:  # Если сосед ещё не посещён
                    distance[neighbor] = distance[current] + 1
                    parent[neighbor] = current
                    queue.append(neighbor)
                elif parent[current] != neighbor:  # Если найден цикл
                    # Вычисляем длину цикла
                    cycle_length = distance[current] + distance[neighbor] + 1
                    if cycle_length < min_cycle_length:
                        min_cycle_length = cycle_length

                        # Восстанавливаем путь цикла
                        path = []
                        temp = current
                        while temp != -1:
                            path.append(temp)
                            temp = parent[temp]
                        path.reverse()

                        temp = neighbor
                        while temp != -1:
                            path.append(temp)
                            temp = parent[temp]

                        min_cycle_path = path

    # Если цикл найден, выводим его
    if min_cycle_length != float('inf'):
        print(*min_cycle_path[:-1])
    else:
        print("NO CYCLES")

# Запускаем функцию
find_shortest_cycle()

# 13. Алгоритм Косарайю выделения компонент сильной связности орграфа.
# **Алгоритм Косарайю для выделения компонент сильной связности (КСС) в орграфе**

## **1. Что такое компоненты сильной связности (КСС)?**
В **ориентированном графе** (орграфе) компонента сильной связности — это максимальный подграф, в котором:
- **Любые две вершины достижимы друг из друга** (существует путь `u → v` и `v → u`).

**Пример**:  
В графе зависимостей КСС помогают находить **группы взаимозависимых модулей**.

---

## **2. Алгоритм Косарайю (Kosaraju-Sharir)**
Алгоритм состоит из **двух проходов DFS**:
1. **Первый DFS** — определяет порядок выхода вершин (обратный топологический порядок).
2. **Транспонирование графа** — разворот всех рёбер.
3. **Второй DFS** — обрабатывает вершины в порядке первого шага, выделяя КСС.

### **Почему работает?**
- Транспонированный граф **сохраняет КСС**, но **меняет достижимость между ними**.
- Обработка вершин в порядке убывания времени выхода гарантирует, что мы сначала посещаем **"корневые" КСС**.

## **4. Пример работы**
**Исходный граф**:
```
0 → 1 → 2 → 0  (цикл: КСС {0, 1, 2})
3 → 4 → 3      (цикл: КСС {3, 4})
5 → 6          (КСС {5}, {6})
```

**Код**:
```python
graph = {
    0: [1],
    1: [2],
    2: [0],
    3: [4],
    4: [3],
    5: [6],
    6: []
}

print(kosaraju(graph))
```

**Вывод**:
```
[[0, 2, 1], [3, 4], [5], [6]]
```


## **5. Визуализация шагов**
### **Шаг 1: Первый DFS**
- Запускаем DFS из вершины `0`:  
  Порядок выхода: `2, 1, 0` (или другой вариативый).
- Затем из `3`:  
  Порядок выхода: `4, 3`.
- Вершины `5` и `6`:  
  Порядок: `6, 5`.

**Итоговый порядок**: `[6, 5, 4, 3, 2, 1, 0]` (пример).

### **Шаг 2: Транспонированный граф**
```
0 → 2 → 1 → 0
3 → 4 → 3
6 → 5
```

### **Шаг 3: Второй DFS**
1. Обрабатываем `0` (первый в обратном порядке):  
   Находим КСС `{0, 1, 2}`.
2. Затем `3`:  
   КСС `{3, 4}`.
3. Затем `5`:  
   КСС `{5}`.
4. Затем `6`:  
   КСС `{6}`.


## **6. Сложность алгоритма**
- **Время**: `O(V + E)` (два прохода DFS + транспонирование).
- **Память**: `O(V + E)` (хранение графа и транспонированного).


**Плюсы Косарайю**:
- Проще реализовать.
- Легче визуализировать.

## **8. Применение**
- Поиск **сильно связанных компонент** в соцсетях (взаимные подписки).
- Анализ **зависимостей в коде** (циклические импорты).
- Оптимизация маршрутов в транспортных сетях.

**Итог**:  
Алгоритм Косарайю — **элегантный способ** выделения КСС за **два прохода DFS**. Он особенно полезен для анализа циклов в орграфах.

In [ ]:
from collections import defaultdict

def kosaraju(graph):
    # Шаг 1: Первый DFS для определения порядка выхода
    visited = set()
    order = []
    
    def dfs_pass1(node):
        stack = [(node, False)]
        while stack:
            current, processed = stack.pop()
            if processed:
                order.append(current)
                continue
            if current in visited:
                continue
            visited.add(current)
            stack.append((current, True))  # Помечаем как обработанную
            for neighbor in graph.get(current, []):
                if neighbor not in visited:
                    stack.append((neighbor, False))

    for node in graph:
        if node not in visited:
            dfs_pass1(node)

    # Шаг 2: Транспонируем граф (разворачиваем рёбра)
    reversed_graph = defaultdict(list)
    for u in graph:
        for v in graph[u]:
            reversed_graph[v].append(u)

    # Шаг 3: Второй DFS в порядке убывания времени выхода
    visited = set()
    components = []
    
    for node in reversed(order):
        if node not in visited:
            stack = [node]
            visited.add(node)
            component = []
            while stack:
                current = stack.pop()
                component.append(current)
                for neighbor in reversed_graph.get(current, []):
                    if neighbor not in visited:
                        visited.add(neighbor)
                        stack.append(neighbor)
            components.append(component)
    
    return components

# 14. Выделение компонент связности обходом в ширину.
# **Выделение компонент связности обходом в ширину (BFS)**

## **1. Основные понятия**
**Компонента связности** в неориентированном графе — это максимальный подграф, в котором:
- Любые две вершины соединены путём.
- Нет рёбер между вершинами из разных компонент.

**Обход в ширину (BFS)** — алгоритм, который посещает все вершины, **достижимые из стартовой**, уровень за уровнем.

---

## **2. Алгоритм выделения компонент связности BFS**
### **Шаги алгоритма**:
1. **Инициализация**:  
   - Создаём массив `visited` для отметки посещённых вершин.  
   - Создаём список `components` для хранения компонент связности.  

2. **Для каждой вершины**:  
   - Если вершина **не посещена**, запускаем из неё BFS.  
   - Все посещённые вершины в этом BFS образуют **новую компоненту связности**.  

3. **BFS для одной компоненты**:  
   - Используем очередь для обхода в ширину.  
   - Помечаем вершины как посещённые и добавляем их в текущую компоненту.  

## **3. Пример работы**
**Граф**:

0 —— 1    3 —— 4
|    |    | 
2    5    6


## **4. Визуализация BFS**
### **Компонента 1 (старт из вершины 0)**:
1. `queue = [0]`, `visited = {0}`, `component = []`  
2. Обрабатываем `0`:  
   - Добавляем `0` в `component`.  
   - Соседи `1` и `2` добавляются в очередь.  
   `queue = [1, 2]`, `visited = {0, 1, 2}`  
3. Обрабатываем `1`:  
   - Добавляем `1` в `component`.  
   - Сосед `5` добавляется в очередь.  
   `queue = [2, 5]`, `visited = {0, 1, 2, 5}`  
4. Обрабатываем `2`:  
   - Добавляем `2` в `component`.  
   - Нет новых соседей.  
   `queue = [5]`  
5. Обрабатываем `5`:  
   - Добавляем `5` в `component`.  
   - Нет новых соседей.  
   `queue = []`  

**Итог**: `component = [0, 1, 2, 5]`  

### **Компонента 2 (старт из вершины 3)**:
Аналогично получаем `[3, 4, 6]`.

---

## **5. Сложность алгоритма**
- **Время**: `O(V + E)` (каждая вершина и ребро обрабатываются ровно один раз).  
- **Память**: `O(V)` (хранение очереди и массива посещений).  

---

## **6. Сравнение BFS и DFS для компонент связности**
| Критерий               | BFS                          | DFS                          |
|------------------------|------------------------------|------------------------------|
| **Используемая структура** | Очередь (`deque`)          | Стек (рекурсия/стек)         |
| **Порядок обхода**      | Уровень за уровнем           | В глубину                    |
| **Применение**          | Короткие пути, кластеры      | Рекурсивные зависимости      |

**Когда выбирать BFS?**  
- Если важно **минимизировать длину путей** (например, поиск кратчайшего пути в компоненте).  
- Если граф **очень широкий** (BFS использует меньше памяти, чем DFS на глубоких графах).  

## **8. Применение**
- **Социальные сети**: Поиск изолированных групп пользователей.  
- **Компьютерные сети**: Обнаружение отключённых подсетей.  
- **Анализ данных**: Кластеризация связанных объектов.  

**Итог**:  
BFS — эффективный способ выделения компонент связности, работающий за линейное время. Он особенно полезен, когда важен **поуровневый обход** или анализ **близости вершин**.

In [ ]:
from collections import deque

def find_connected_components_bfs(graph):
    visited = set()
    components = []
    
    for node in graph:
        if node not in visited:
            # Очередь для BFS
            queue = deque([node])
            visited.add(node)
            component = []
            
            while queue:
                current = queue.popleft()
                component.append(current)
                
                for neighbor in graph[current]:
                    if neighbor not in visited:
                        visited.add(neighbor)
                        queue.append(neighbor)
            
            components.append(component)
    
    return components

graph = {
    0: [1, 2],
    1: [0, 5],
    2: [0],
    3: [4, 6],
    4: [3],
    5: [1],
    6: [3]
}

print(find_connected_components_bfs(graph))

def is_connected(graph):
    components = find_connected_components_bfs(graph)
    return len(components) == 1

print(is_connected(graph))  # False (граф из примера несвязный)

# 15. Нахождение кратчайшего цикла в невзвешенном графе.

In [ ]:
from collections import deque

# Ввод данных
n, m, x, y = map(int, input().split())  # Количество вершин, рёбер, начальная и конечная вершины
graph = [[] for _ in range(n)]          # Создаём список смежности

# Считываем рёбра и строим граф
for _ in range(m):
    u, v = map(int, input().split())
    graph[u].append(v)  # Добавляем ребро u -> v
    graph[v].append(u)  # Добавляем ребро v -> u (граф неориентированный)

# Обход в ширину (BFS)
def bfs(start, end):
    queue = deque([start])               # Очередь для BFS
    distance = [-1] * n                  # Массив расстояний, инициализируем -1 (не посещено)
    distance[start] = 0                  # Расстояние до стартовой вершины равно 0

    while queue:
        current = queue.popleft()         # Берём вершину из очереди

        # Если достигли целевой вершины, возвращаем расстояние
        if current == end:
            return distance[current]

        # Перебираем всех соседей текущей вершины
        for neighbor in graph[current]:
            if distance[neighbor] == -1:  # Если сосед ещё не посещён
                distance[neighbor] = distance[current] + 1  # Обновляем расстояние
                queue.append(neighbor)    # Добавляем соседа в очередь

    # Если конечная вершина недостижима, возвращаем -1
    return -1

# Запускаем BFS и выводим результат
result = bfs(x, y)
print(result)

# 16. Алгоритм Дейкстры (наивная реализация).
Время: O(V² + E) (каждый из V шагов требует поиска минимума за O(V)).

Память: O(V) (хранение расстояний и посещённых вершин).

In [ ]:
import sys

def dijkstra_naive(graph, start):
    # Инициализация
    n = len(graph)
    visited = [False] * n
    distance = [sys.maxsize] * n  # Изначально все расстояния "бесконечны"
    distance[start] = 0  # Расстояние до стартовой вершины = 0

    for _ in range(n):
        # Находим вершину с минимальным расстоянием среди непосещённых
        min_dist = sys.maxsize
        u = -1
        for v in range(n):
            if not visited[v] and distance[v] < min_dist:
                min_dist = distance[v]
                u = v

        if u == -1:  # Если все вершины посещены или недостижимы
            break

        visited[u] = True

        # Обновляем расстояния до соседей u
        for v, weight in graph[u]:
            if not visited[v] and distance[u] + weight < distance[v]:
                distance[v] = distance[u] + weight

    return distance

# 17. Алгоритм Дейкстры с кучей.

Работает только с неотрицательными весами (как и должен алгоритм Дейкстры)

Использует жадную стратегию - всегда обрабатывает ближайшую доступную вершину

Сложность: O(E + V log V) благодаря использованию кучи

In [ ]:
from random import randint

def read_graph():
    """Чтение графа из входных данных и создание его представления в виде словаря смежности"""
    G = {}  # Создаем пустой словарь для хранения графа
    n, m = map(int, input().split())  # Читаем количество вершин (n) и ребер (m)
    
    for _ in range(m):  # Читаем каждое ребро
        u, v = input().split()  # Читаем две соединенные вершины
        
        # Добавляем вершины в граф, если их еще нет
        if u not in G:
            G[u] = {}  # Для вершины u создаем словарь смежных вершин
        if v not in G:
            G[v] = {}  # Для вершины v создаем словарь смежных вершин
            
        w = randint(1, 100)  # Генерируем случайный вес ребра от 1 до 100
        G[u][v] = w  # Добавляем ребро u -> v с весом w
        G[v][u] = w  # Добавляем ребро v -> u с тем же весом (граф неориентированный)

    return G  # Возвращаем полученный граф


from heapq import heappush, heappop  # Импортируем функции для работы с кучей (приоритетной очередью)

def deykstra(G: dict, start: str):
    """Реализация алгоритма Дейкстры для поиска кратчайших путей от начальной вершины"""
    h = []  # Инициализируем кучу (приоритетную очередь)
    heappush(h, (0, start))  # Добавляем начальную вершину с расстоянием 0
    
    # Инициализируем расстояния до всех вершин как бесконечность
    dist = {v: float('+inf') for v in G}
    dist[start] = 0  # Расстояние до начальной вершины = 0
    
    used = set()  # Множество для отслеживания посещенных вершин

    while len(h):  # Пока в куче есть элементы
        w, v = heappop(h)  # Извлекаем вершину с минимальным расстоянием
        
        # Проверяем всех соседей текущей вершины
        for neighbor in G[v]:
            # Если найден более короткий путь до соседа
            if w + G[v][neighbor] < dist[neighbor]:
                dist[neighbor] = w + G[v][neighbor]  # Обновляем расстояние
                heappush(h, (dist[neighbor], neighbor))  # Добавляем в кучу с новым расстоянием
        
        used.add(v)  # Помечаем вершину как посещенную
    
    return dist  # Возвращаем словарь с кратчайшими расстояниями до всех вершин

# 18. Алгоритм Флойда-Уоршелла.
# **Алгоритм Флойда-Уоршелла**

## **1. Назначение алгоритма**
Алгоритм Флойда-Уоршелла находит **кратчайшие пути между всеми парами вершин** в взвешенном графе. Работает с:
- Отрицательными весами рёбер (но без отрицательных циклов)
- Ориентированными и неориентированными графами

## **2. Основная идея**
Динамическое программирование: последовательно улучшаем оценки кратчайших путей, рассматривая все возможные промежуточные вершины.

### **Матрица расстояний**
- `dist[i][j]` — длина кратчайшего пути из `i` в `j`
- Изначально:
  - `dist[i][j] = 0`, если `i == j`
  - `dist[i][j] = вес ребра i→j`, если ребро существует
  - `dist[i][j] = +∞`, если пути нет

## **4. Пример работы**
**Граф**:
```
0 → 1 (вес 3)
1 → 2 (вес 1)
2 → 0 (вес -2)
```

**Матрица расстояний**:
```
Исходная:
    0   1   2
0 [ 0,  3, ∞ ]
1 [ ∞, 0, 1 ]
2 [-2, ∞, 0 ]

После k=0 (промежуточная 0):
   Обновляем пути через 0, например 2→0→1: -2 + 3 = 1 < ∞

После k=1 (промежуточная 1):
   Обновляем пути через 1, например 0→1→2: 3 + 1 = 4 < ∞

После k=2 (промежуточная 2):
   Обновляем пути через 2, например 1→2→0: 1 + (-2) = -1 < ∞

Итоговая:
    0   1   2
0 [ 0,  3, 4 ]
1 [ -1, 0, 1 ]
2 [ -2, 1, 0 ]
```

## **5. Обнаружение отрицательных циклов**
Добавим проверку после основного алгоритма:
```python
for i in range(n):
    if dist[i][i] < 0:  # Есть путь i→...→i с отрицательным весом
        print("Граф содержит отрицательный цикл")
        break
```

## **6. Сложность**
- **Время**: O(V³) — три вложенных цикла по количеству вершин
- **Память**: O(V²) — хранение матрицы расстояний

## **7. Применение**
- Анализ транспортных сетей
- Маршрутизация в компьютерных сетях
- Решение задач оптимизации

## **8. Сравнение с другими алгоритмами**
| Алгоритм          | Отрицательные веса | Отрицательные циклы | Сложность  | Применение               |
|-------------------|--------------------|---------------------|------------|--------------------------|
| Флойд-Уоршелл     | Да                 | Обнаруживает        | O(V³)      | Все пары вершин          |
| Дейкстра          | Нет                | Нет                 | O(E+VlogV) | Одна стартовая вершина   |
| Беллман-Форд      | Да                 | Обнаруживает        | O(VE)      | Одна стартовая вершина   |

**Итог**: Алгоритм Флойда-Уоршелла — универсальное решение для поиска кратчайших путей между всеми парами вершин, особенно полезное при работе с графами, содержащими отрицательные веса.

In [ ]:
def floyd_warshall(graph):
    n = len(graph)
    
    # Инициализация матрицы расстояний
    dist = [[float('inf')] * n for _ in range(n)]
    for i in range(n):
        dist[i][i] = 0
    for u in graph:
        for v in graph[u]:
            dist[u][v] = graph[u][v]
    
    # Основная часть алгоритма
    for k in range(n):           # Промежуточная вершина
        for i in range(n):       # Начальная вершина
            for j in range(n):   # Конечная вершина
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    
    return dist

# 19. Алгоритм Форда-Беллмана.
# **Алгоритм Форда-Беллмана**

## **1. Назначение алгоритма**
Алгоритм Форда-Беллмана находит **кратчайшие пути от одной стартовой вершины до всех остальных** в взвешенном графе. Ключевые особенности:
- Работает с **отрицательными весами** рёбер
- Обнаруживает **отрицательные циклы**, достижимые из стартовой вершины
- Применим для **ориентированных** и **неориентированных** графов (в неориентированном случае отрицательные рёбра автоматически создают отрицательные циклы)

## **2. Основная идея**
Алгоритм релаксации: последовательно улучшает оценки кратчайших путей, рассматривая все рёбра графа. После `V-1` итераций гарантированно находит оптимальные пути (если нет отрицательных циклов).

## **4. Пример работы**

**Граф**:
```
0 → 1 (вес 4)
0 → 2 (вес 3)
1 → 3 (вес -2)
2 → 1 (вес -1)
3 → 2 (вес 1)
```

**Выполнение**:
1. Инициализация: `dist = [0, ∞, ∞, ∞]`
2. После 1-й итерации: `[0, 4, 3, ∞]`
3. После 2-й итерации: `[0, 2, 3, 2]`
4. После 3-й итерации: `[0, 2, 3, 0]`

## **5. Обнаружение отрицательных циклов**
Алгоритм выполняет дополнительную проверку после `V-1` итераций:
- Если на `V`-й итерации происходит обновление расстояний → существует отрицательный цикл

## **6. Сложность**
- **Время**: O(V×E) — в худшем случае (V-1) полных проходов по всем рёбрам
- **Память**: O(V) — хранение расстояний до вершин

## **7. Сравнение с другими алгоритмами**

| Критерий          | Форд-Беллман | Дейкстра | Флойд-Уоршелл |
|-------------------|--------------|----------|---------------|
| Отрицательные веса | Да           | Нет      | Да            |
| Отрицательные циклы| Обнаруживает | Нет      | Обнаруживает  |
| Сложность         | O(V×E)       | O(E+VlogV)| O(V³)        |
| Применение        | Одна вершина | Одна вершина | Все пары    |

## **8. Оптимизации**
1. **Досрочный выход**: если на итерации не было изменений
2. **Очередь для хранения изменённых вершин** (аналог алгоритма SPFA)

## **9. Применение**
- Финансовые системы (арбитражные возможности)
- Маршрутизация в сетях
- Анализ транспортных потоков

**Итог**: Алгоритм Форда-Беллмана — гибкое решение для задач с отрицательными весами, хотя и менее эффективное, чем Дейкстра для графов без отрицательных весов.

In [ ]:
def ford_bellman(graph, start):
    # Инициализация расстояний
    dist = {v: float('inf') for v in graph}
    dist[start] = 0
    edges = []
    
    # Составляем список всех рёбер
    for u in graph:
        for v, w in graph[u].items():
            edges.append((u, v, w))
    
    # Основная часть алгоритма
    for _ in range(len(graph) - 1):
        updated = False
        for u, v, w in edges:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                updated = True
        if not updated:  # Досрочный выход если нет изменений
            break
    
    # Проверка на отрицательные циклы
    for u, v, w in edges:
        if dist[u] + w < dist[v]:
            raise ValueError("Граф содержит отрицательный цикл")
    
    return dist

# 20. Остовные деревья. Алгоритм Прима.
### Остовные деревья и алгоритм Прима

**Остовное дерево** (или минимальное остовное дерево) — это подмножество рёбер связного неориентированного графа, которое соединяет все вершины графа, не образуя циклов, и имеет минимальную возможную сумму весов рёбер.

**Алгоритм Прима** — это жадный алгоритм для нахождения минимального остовного дерева в графе. Он работает следующим образом:

1. Начинаем с произвольной вершины графа и добавляем её в остовное дерево.
2. На каждом шаге выбираем рёбер, которое соединяет уже добавленные в остовное дерево вершины с вершинами, которые ещё не добавлены, и имеет минимальный вес.
3. Добавляем эту вершину и соответствующее ребро в остовное дерево.
4. Повторяем шаги 2 и 3, пока все вершины не будут добавлены в остовное дерево.

### Объяснение кода

1. **Граф** представлен в виде словаря, где ключи — это вершины, а значения — это словари соседей с весами рёбер.
2. **Куча** (min_heap) используется для хранения рёбер, которые могут быть добавлены в остовное дерево, с их весами.
3. **Цикл** продолжается, пока есть рёбра в куче. Если текущая вершина уже посещена, она пропускается.
4. Если вершина не посещена, она добавляется в остовное дерево, и все её соседние рёбра добавляются в кучу, если соседние вершины ещё не посещены.
5. В конце выводится список рёбер минимального остовного дерева.

Этот алгоритм работает за \(O(E \log V)\), где \(E\) — количество рёбер, а \(V\) — количество вершин в графе.

In [ ]:
import heapq

def prim(graph, start):
    # Инициализация
    mst = []  # Список рёбер остовного дерева
    visited = set()  # Множество посещённых вершин
    min_heap = [(0, start, None)]  # (вес, текущая вершина, предыдущая вершина)

    while min_heap:
        weight, current_vertex, from_vertex = heapq.heappop(min_heap)

        if current_vertex in visited:
            continue

        visited.add(current_vertex)
        if from_vertex is not None:
            mst.append((from_vertex, current_vertex, weight))

        for neighbor, edge_weight in graph[current_vertex].items():
            if neighbor not in visited:
                heapq.heappush(min_heap, (edge_weight, neighbor, current_vertex))

    return mst

# Пример графа в виде словаря
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

# Запуск алгоритма Прима
start_vertex = 'A'
mst = prim(graph, start_vertex)

# Вывод результата
print("Рёбра минимального остовного дерева:")
for from_vertex, to_vertex, weight in mst:
    print(f"{from_vertex} - {to_vertex}: {weight}")


# 21. Остовные деревья. Алгоритм Краскала. Система непересекающихся множеств для оптимизации алгоритма.
### Остовные деревья и алгоритм Краскала

**Остовное дерево** (или минимальное остовное дерево) — это подмножество рёбер связного неориентированного графа, которое соединяет все вершины графа, не образуя циклов, и имеет минимальную возможную сумму весов рёбер.

**Алгоритм Краскала** — это жадный алгоритм для нахождения минимального остовного дерева, который работает следующим образом:

1. Сначала сортируем все рёбра графа по возрастанию их веса.
2. Инициализируем пустое остовное дерево.
3. Проходим по отсортированным рёбрам и добавляем каждое ребро в остовное дерево, если оно не образует цикл с уже добавленными рёбрами.
4. Для проверки, образует ли ребро цикл, используем структуру данных "система непересекающихся множеств" (Union-Find).

### Система непересекающихся множеств (Union-Find)

Система непересекающихся множеств — это структура данных, которая поддерживает операции объединения и поиска. Она позволяет эффективно управлять множествами и проверять, принадлежат ли два элемента одному множеству.

Основные операции:
- **Find**: находит корень (представителя) множества, к которому принадлежит элемент.
- **Union**: объединяет два множества.

Для оптимизации этих операций часто используются два метода:
- **Сжатие пути** (Path Compression) в операции Find.
- **Объединение по рангу** (Union by Rank) в операции Union.

### Объяснение кода

1. **Класс `UnionFind`** реализует систему непересекающихся множеств с методами `find` и `union`.
2. **Функция `kruskal`**:
   - Создаёт список рёбер из графа и сортирует его по весу.
   - Инициализирует экземпляр `UnionFind` для управления множествами.
   - Проходит по отсорт

In [ ]:
class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [1] * size

    def find(self, p):
        if self.parent[p] != p:
            self.parent[p] = self.find(self.parent[p])  # Сжатие пути
        return self.parent[p]

    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)

        if rootP != rootQ:
            # Объединение по рангу
            if self.rank[rootP] > self.rank[rootQ]:
                self.parent[rootQ] = rootP
            elif self.rank[rootP] < self.rank[rootQ]:
                self.parent[rootP] = rootQ
            else:
                self.parent[rootQ] = rootP
                self.rank[rootP] += 1

def kruskal(graph):
    # Список рёбер (вес, вершина1, вершина2)
    edges = []
    for u in graph:
        for v, weight in graph[u].items():
            edges.append((weight, u, v))
    
    # Сортируем рёбра по весу
    edges.sort()

    uf = UnionFind(len(graph))
    mst = []  # Список рёбер остовного дерева
    total_weight = 0

    for weight, u, v in edges:
        if uf.find(u) != uf.find(v):  # Если u и v не в одном множестве
            uf.union(u, v)  # Объединяем множества
            mst.append((u, v, weight))
            total_weight += weight

    return mst, total_weight

# Пример графа в виде словаря
graph = {
    0: {1: 4, 2: 1},
    1: {0: 4, 2: 2, 3: 5},
    2: {0: 1, 1: 2, 3: 8},
    3: {1: 5, 2: 8}
}

# Запуск алгоритма Краскала
mst, total_weight = kruskal(graph)

# Вывод результата
print("Рёбра минимального остовного дерева:")
for u, v, weight in mst:
    print(f"{u} - {v}: {weight}")
print(f"Общий вес остовного дерева: {total_weight}")

# 22. Игры на ациклических графах. Решение поиском в глубину.

Игры на ациклических графах — это интересная область, в которой можно применять алгоритмы поиска в глубину (DFS) для анализа различных игровых ситуаций. Ациклические графы, такие как ориентированные ациклические графы (DAG), часто используются для моделирования игр, где нет циклов, и каждое состояние может быть достигнуто только одним способом.

### Пример игры на ациклическом графе

Рассмотрим простую игру, в которой игроки перемещаются по вершинам графа, и цель состоит в том, чтобы достичь определённой вершины (например, "выигрышной" вершины). Игроки могут перемещаться только по рёбрам графа, и игра заканчивается, когда один из игроков достигает выигрышной вершины.

### Решение с помощью поиска в глубину (DFS)

Поиск в глубину может быть использован для анализа всех возможных путей в игре. Мы можем определить, является ли текущее состояние выигрышным или проигрышным для игрока, основываясь на состоянии его соседей.

#### Алгоритм

1. Начинаем с текущей вершины.
2. Если текущая вершина — выигрышная, возвращаем "выигрыш".
3. Если все соседние вершины являются выигрышными для противника, то текущая вершина — проигрышная.
4. Если хотя бы одна соседняя вершина является проигрышной для противника, то текущая вершина — выигрышная.

### Объяснение кода

1. **Функция `dfs`**:
   - Принимает граф, текущую вершину и словарь `memo` для хранения уже вычисленных результатов.
   - Проверяет, является ли текущая вершина выигрышной или проигрышной, используя рекурсивный подход.
   - Если текущая вершина — "win", возвращает `True`.
   - Если все соседи являются выигрышными для противника, текущая вершина считается проигрышной.

2. **Граф** представлен в виде словаря, где ключи — это вершины, а значения — списки соседей.

3. **Запуск DFS** начинается с заданной стартовой вершины, и результат выводится на экран.

### Заключение

Использование поиска в глубину для анализа игр на ациклических графах позволяет эффективно определять выигрышные и проигрышные состояния. Этот подход может быть расширен для более сложных игр и стратегий, а также для анализа различных игровых ситуаций.

In [ ]:
def dfs(graph, vertex, memo):
    # Если уже вычислено, возвращаем результат
    if vertex in memo:
        return memo[vertex]

    # Если текущая вершина — выигрышная
    if vertex == 'win':
        return True

    # Проверяем соседние вершины
    for neighbor in graph[vertex]:
        if not dfs(graph, neighbor, memo):  # Если сосед проигрывает
            memo[vertex] = True  # Текущая вершина выигрывает
            return True

    memo[vertex] = False  # Все соседи выигрывают, текущая проигрывает
    return False

# Пример графа
graph = {
    'A': ['B', 'C'],
    'B': ['D'],
    'C': ['D'],
    'D': ['win'],
    'E': ['F'],
    'F': ['win'],
    'win': []
}

# Запуск DFS для проверки, является ли 'A' выигрышной вершиной
memo = {}
start_vertex = 'A'
is_winning = dfs(graph, start_vertex, memo)

# Вывод результата
if is_winning:
    print(f"Вершина '{start_vertex}' является выигрышной.")
else:
    print(f"Вершина '{start_vertex}' является проигрышной.")

# 23. Сумма игр. Функция Шпрага-Гранди.

### Сумма игр и функция Шпрага-Гранди

**Сумма игр** — это концепция в теории игр, которая позволяет анализировать ситуации, когда игроки могут выбирать из нескольких независимых игр. Если у нас есть несколько игр, то результат (выигрыш или проигрыш) зависит от состояния каждой из этих игр.

**Функция Шпрага-Гранди** (или просто функция Гранди) используется для анализа игр с нулевой суммой, чтобы определить, является ли данная позиция выигрышной или проигрышной. Она основана на принципе, что позиция является выигрышной, если существует хотя бы один ход, который переводит её в проигрышную позицию для противника.

### Определение функции Гранди

1. **Функция Гранди** для позиции \( G \) определяется как минимальное неотрицательное целое число, которое не может быть получено из значений функции Гранди для всех возможных ходов из позиции \( G \). Это значение называется **mex** (minimum excludant).

2. Если функция Гранди для позиции равна 0, то это проигрышная позиция для текущего игрока. Если она больше 0, то это выигрышная позиция.

### Пример игры

Рассмотрим простую игру, в которой игроки могут удалять 1, 2 или 3 камня из кучи. Цель состоит в том, чтобы заставить противника оказаться в позиции, где он не может сделать ход.

### Объяснение кода

1. **Функция `grundy(n, memo)`**:
   - Принимает количество камней \( n \) и словарь `memo` для хранения уже вычисленных значений функции Гранди.
   - Если значение уже вычислено, оно возвращается из `memo`.
   - Для каждого возможного хода (удаление 1, 2 или 3 камней) вычисляется функция Гранди для новой позиции.
   - Собирается множество `reachable`, которое содержит значения функции Гранди для всех возможных ходов.
   - Находится минимальное неотрицательное целое число, которое не содержится в `reachable` (mex), и сохраняется в `memo`.

2. **Функция `is_winning_position(n)`**:
   - Вызывает функцию `grundy` и проверяет, является ли позиция выигрышной (функция Гранди не равна 0).

### Заключение

Функция Шпрага-Гранди является мощным инструментом для анализа игр с нулевой суммой. Она позволяет определить, является ли данная позиция выигрышной или проигрышной, и может быть применена к различным играм, включая более сложные варианты.

In [ ]:
def grundy(n, memo):
    if n in memo:
        return memo[n]
    
    reachable = set()
    for move in range(1, 4):  # Игрок может удалить 1, 2 или 3 камня
        if n - move >= 0:
            reachable.add(grundy(n - move, memo))
    
    # Находим минимальное неотрицательное целое число, которое не содержится в reachable
    mex = 0
    while mex in reachable:
        mex += 1
    
    memo[n] = mex
    return mex

def is_winning_position(n):
    memo = {}
    return grundy(n, memo) != 0

# Пример использования
n = 10  # Количество камней
if is_winning_position(n):
    print(f"Позиция с {n} камнями является выигрышной.")
else:
    print(f"Позиция с {n} камнями является проигрышной.")


# 24. Игры на произвольных графах.

In [ ]:
from collections import deque

def determine_winner(N, M, K, edges):
    # Строим обратный граф (куда ведут рёбра)
    adj = [[] for _ in range(N)]
    reverse_adj = [[] for _ in range(N)]
    out_degree = [0] * N
    
    for u, v in edges:
        adj[u].append(v)
        reverse_adj[v].append(u)
        out_degree[u] += 1
    
    # Инициализация: вершины без исходящих рёбер — LOSE
    status = [None] * N
    queue = deque()
    
    for u in range(N):
        if out_degree[u] == 0:
            status[u] = 'LOSE'
            queue.append(u)
    
    # Обратный BFS для пометок вершин
    while queue:
        u = queue.popleft()
        
        for v in reverse_adj[u]:
            if status[v] is not None:
                continue
            
            # Если хотя бы один переход ведёт в LOSE, вершина — WIN
            if status[u] == 'LOSE':
                status[v] = 'WIN'
                queue.append(v)
            else:
                # Уменьшаем счётчик исходящих рёбер
                out_degree[v] -= 1
                if out_degree[v] == 0:
                    status[v] = 'LOSE'
                    queue.append(v)
    
    if status[K] == 'WIN':
        return "First"
    elif status[K] == 'LOSE':
        return "Second"
    else:
        return "Draw"

# Чтение входных данных
N, M, K = map(int, input().split())
edges = [tuple(map(int, input().split())) for _ in range(M)]

# Определение победителя
print(determine_winner(N, M, K, edges))

# 25. ООП, инкапсуляция, наследование, полиморфизм.
Объектно-ориентированное программирование (ООП) — это парадигма программирования, основанная на концепции "объектов", которые могут содержать данные и код: данные в виде полей (или атрибутов), а код в виде процедур (или методов). Основные принципы ООП включают инкапсуляцию, наследование и полиморфизм.

### 1. Инкапсуляция

**Инкапсуляция** — это принцип, который позволяет скрыть внутренние детали реализации объекта и предоставить доступ к ним только через определённые методы. Это помогает защитить данные от некорректного использования и упрощает взаимодействие с объектом.

Пример на Python:

```python
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.__balance = balance  # Приватное поле

    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            print(f"Deposited {amount}. New balance: {self.__balance}")
        else:
            print("Deposit amount must be positive.")

    def withdraw(self, amount):
        if 0 < amount <= self.__balance:
            self.__balance -= amount
            print(f"Withdrew {amount}. New balance: {self.__balance}")
        else:
            print("Invalid withdrawal amount.")

    def get_balance(self):
        return self.__balance

# Пример использования
account = BankAccount("Alice", 100)
account.deposit(50)
account.withdraw(30)
print(account.get_balance())
```

### 2. Наследование

**Наследование** — это механизм, который позволяет создавать новый класс на основе существующего. Новый класс (наследник) наследует свойства и методы родительского класса (базового класса), что позволяет повторно использовать код и расширять функциональность.

Пример на Python:

```python
class Animal:
    def speak(self):
        raise NotImplementedError("Subclasses must implement this method.")

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

# Пример использования
animals = [Dog(), Cat()]
for animal in animals:
    print(animal.speak())
```

### 3. Полиморфизм

**Полиморфизм** — это способность объектов разных классов обрабатывать данные по одному и тому же интерфейсу. Это позволяет использовать один и тот же метод для объектов разных классов, что упрощает код и делает его более гибким.

Пример на Python:

```python
class Shape:
    def area(self):
        raise NotImplementedError("Subclasses must implement this method.")

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius ** 2

# Пример использования
shapes = [Rectangle(5, 10), Circle(7)]
for shape in shapes:
    print(f"Area: {shape.area()}")
```

### Заключение

Объектно-ориентированное программирование предоставляет мощные инструменты для организации и структурирования кода. Инкапсуляция, наследование и полиморфизм — это ключевые концепции, которые помогают создавать гибкие, поддерживаемые и расширяемые программы. Эти принципы позволяют разработчикам эффективно управлять сложностью программного обеспечения и улучшать его качество.

# 26. Классы в Python. Магические методы классов.
В Python классы являются основным строительным блоком объектно-ориентированного программирования. Они позволяют создавать объекты, которые могут содержать как данные (атрибуты), так и методы (функции, связанные с объектом). 

### Магические методы классов

**Магические методы** (или специальные методы) — это методы, которые имеют специальные имена и позволяют определять поведение объектов класса в различных ситуациях. Они начинаются и заканчиваются двойными подчеркиваниями (`__`). Вот некоторые из наиболее часто используемых магических методов:

1. **`__init__`**: Конструктор класса, вызывается при создании нового объекта.
2. **`__str__`**: Определяет поведение функции `str()` и функции `print()`, возвращая строковое представление объекта.
3. **`__repr__`**: Определяет поведение функции `repr()`, возвращая официальное строковое представление объекта, которое можно использовать для его воссоздания.
4. **`__len__`**: Определяет поведение функции `len()`, возвращая длину объекта.
5. **`__getitem__`**: Позволяет использовать квадратные скобки для доступа к элементам объекта (например, `obj[key]`).
6. **`__setitem__`**: Позволяет использовать квадратные скобки для установки значений (например, `obj[key] = value`).
7. **`__delitem__`**: Позволяет использовать квадратные скобки для удаления элементов (например, `del obj[key]`).
8. **`__add__`**: Определяет поведение оператора `+`.
9. **`__sub__`**: Определяет поведение оператора `-`.
10. **`__eq__`**: Определяет поведение оператора `==`.

### Заключение

Магические методы в Python позволяют создавать более интуитивно понятные и удобные для использования классы. Они позволяют определять поведение объектов в различных ситуациях, таких как арифметические операции, сравнения и преобразования в строку. Использование магических методов делает код более чистым и понятным, а также улучшает взаимодействие с объектами классов.

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"Vector({self.x}, {self.y})"

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __len__(self):
        return int((self.x**2 + self.y**2) ** 0.5)

# Пример использования
v1 = Vector(2, 3)
v2 = Vector(4, 5)

print(v1)  # Вывод: Vector(2, 3)
print(v2)  # Вывод: Vector(4, 5)

v3 = v1 + v2
print(v3)  # Вывод: Vector(6, 8)

v4 = v2 - v1
print(v4)  # Вывод: Vector(2, 2)

print(v1 == v2)  # Вывод: False
print(len(v1))   # Вывод: 3

# 27. Абстрактные классы, модуль abc.
Абстрактные классы в Python — это классы, которые не могут быть инстанцированы напрямую и предназначены для определения интерфейса для других классов. Они позволяют задавать методы, которые должны быть реализованы в дочерних классах, обеспечивая тем самым определённый уровень абстракции.

Для создания абстрактных классов в Python используется модуль `abc` (Abstract Base Classes). Этот модуль предоставляет инструменты для определения абстрактных классов и абстрактных методов.

### Основные компоненты модуля `abc`

1. **`ABC`**: Базовый класс для определения абстрактных классов.
2. **`abstractmethod`**: Декоратор, который используется для определения абстрактных методов.

### Объяснение кода

1. **Импортирование модуля**: Мы импортируем `ABC` и `abstractmethod` из модуля `abc`.

2. **Определение абстрактного класса**: Класс `Shape` наследует от `ABC` и содержит два абстрактных метода: `area` и `perimeter`. Эти методы не имеют реализации и должны быть реализованы в дочерних классах.

3. **Создание дочерних классов**: Классы `Rectangle` и `Circle` наследуют от `Shape` и реализуют абстрактные методы `area` и `perimeter`.

4. **Использование классов**: Мы создаем список фигур и для каждой фигуры вызываем методы `area` и `perimeter`, которые возвращают соответствующие значения.

### Преимущества использования абстрактных классов

- **Определение интерфейса**: Абстрактные классы позволяют определить интерфейс, который должны реализовать все дочерние классы, что обеспечивает согласованность.
- **Упрощение кода**: Они помогают избежать дублирования кода, так как общая логика может быть реализована в абстрактном классе.
- **Улучшение читаемости**: Код становится более понятным, так как явно указывает, какие методы должны быть реализованы.

### Заключение

Абстрактные классы и модуль `abc` в Python предоставляют мощные инструменты для создания интерфейсов и определения структуры классов. Они помогают организовать код, обеспечивая его гибкость и поддерживаемость. Использование абстрактных классов позволяет разработчикам создавать более чистые и понятные архитектуры программного обеспечения.

In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

    @abstractmethod
    def perimeter(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def perimeter(self):
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius ** 2

    def perimeter(self):
        return 2 * 3.14 * self.radius

# Пример использования
shapes = [Rectangle(3, 4), Circle(5)]

for shape in shapes:
    print(f"Area: {shape.area()}, Perimeter: {shape.perimeter()}")

# 28. Генераторы и сопроцессы
Генераторы и сопроцессы — это мощные инструменты в Python, которые позволяют работать с итерациями и асинхронным программированием. Давайте рассмотрим каждую из этих концепций подробнее.

### Генераторы

**Генераторы** — это специальные функции, которые позволяют создавать итераторы. Они используют ключевое слово `yield` для возврата значений по одному за раз, сохраняя состояние функции между вызовами. Это позволяет экономить память, так как значения генерируются по мере необходимости, а не создаются все сразу.

### Объяснение кода

1. **Определение генератора**: Функция `fibonacci` определяет генератор, который генерирует первые `n` чисел Фибоначчи.
2. **Использование `yield`**: Вместо `return` используется `yield`, что позволяет функции возвращать значение и сохранять своё состояние.
3. **Итерация**: При итерации по генератору значения генерируются по одному, что позволяет экономить память.

### Сопроцессы

**Сопроцессы** (или корутины) — это более сложная концепция, которая позволяет выполнять несколько функций одновременно, при этом они могут приостанавливать своё выполнение и передавать управление другим функциям. В Python сопроцессы реализуются с помощью `async` и `await`.

#### Пример сопроцессов

Вот пример использования сопроцессов для асинхронного выполнения:

### Объяснение кода

1. **Определение сопроцесса**: Функция `say_hello` определена как асинхронная с помощью ключевого слова `async`.
2. **Приостановка выполнения**: Внутри функции используется `await asyncio.sleep(1)`, чтобы приостановить выполнение на 1 секунду, позволяя другим задачам выполняться в это время.
3. **Запуск сопроцессов**: Функция `main` также является асинхронной и вызывает `say_hello`. Для запуска сопроцессов используется `asyncio.run(main())`.

### Сравнение генераторов и сопроцессов

- **Генераторы**: Используются для создания итераторов, которые возвращают значения по одному. Они не могут приостанавливать выполнение и возвращать управление другим функциям.
- **Сопроцессы**: Позволяют выполнять несколько функций одновременно, приостанавливая выполнение и передавая управление другим функциям. Они идеально подходят для асинхронного программирования.

### Заключение

Генераторы и сопроцессы — это мощные инструменты в Python, которые позволяют эффективно управлять итерациями и асинхронным выполнением. Генераторы помогают экономить память и упрощают работу с последовательностями, в то время как сопроцессы позволяют реализовывать асинхронное программирование, что особенно полезно в приложениях, требующих высокой производительности и отзывчивости.

In [ ]:
def range(start=0, stop=None, step=1):
    if stop is None:
        stop = start
        start = 0

    if step > 0:
        while start < stop:
            yield start
            start += step
    elif step < 0: 
        while start > stop:
            yield start
            start += step

def enumerate(Iterated, start_index=0):
    for el in Iterated:
        yield start_index, el
        start_index += 1

def zip(*args):
    min_length = min(len(arg) for arg in args)
    for i in range(min_length):
        yield tuple(arg[i] for arg in args)

def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

# Пример использования
for num in fibonacci(10):
    print(num)

import asyncio

async def say_hello():
    print("Hello")
    await asyncio.sleep(1)  # Приостановка выполнения на 1 секунду
    print("World")

async def main():
    await say_hello()

# Запуск сопроцессов
asyncio.run(main())

# 29. Декораторы
Декораторы в Python — это мощный инструмент, который позволяет изменять или расширять поведение функций или методов без изменения их кода. Декораторы представляют собой функции, которые принимают другую функцию в качестве аргумента и возвращают новую функцию, которая обычно добавляет некоторую функциональность.

### Основы декораторов

Декораторы часто используются для:

- Логирования
- Измерения времени выполнения
- Проверки прав доступа
- Кэширования результатов

### Простой пример декоратора

Вот простой пример декоратора, который добавляет логирование к функции:

```python
def my_decorator(func):
    def wrapper():
        print("Что-то происходит перед вызовом функции.")
        func()
        print("Что-то происходит после вызова функции.")
    return wrapper

@my_decorator
def say_hello():
    print("Привет!")

# Вызов функции
say_hello()
```

### Объяснение кода

1. **Определение декоратора**: Функция `my_decorator` принимает функцию `func` в качестве аргумента и определяет внутреннюю функцию `wrapper`, которая добавляет дополнительное поведение.
2. **Использование `@`**: Декоратор применяется к функции `say_hello` с помощью символа `@`. Это эквивалентно вызову `say_hello = my_decorator(say_hello)`.
3. **Вызов функции**: При вызове `say_hello()` фактически вызывается `wrapper()`, который добавляет логирование до и после вызова оригинальной функции.

### Декораторы с аргументами

Декораторы могут также принимать аргументы. Для этого нужно создать дополнительный уровень вложенности:

```python
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                func(*args, **kwargs)
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Привет, {name}!")

# Вызов функции
greet("Алиса")
```

### Объяснение кода

1. **Декоратор с аргументами**: Функция `repeat` принимает аргумент `num_times` и возвращает декоратор `decorator_repeat`.
2. **Внутренний `wrapper`**: Внутренний `wrapper` вызывает оригинальную функцию `func` указанное количество раз.
3. **Применение декоратора**: Декоратор применяется к функции `greet`, и при её вызове она будет выполнена трижды.

### Использование `functools.wraps`

При создании декораторов важно сохранять метаданные оригинальной функции, такие как имя и документация. Для этого можно использовать `functools.wraps`:

```python
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Что-то происходит перед вызовом функции.")
        result = func(*args, **kwargs)
        print("Что-то происходит после вызова функции.")
        return result
    return wrapper

@my_decorator
def say_hello():
    """Функция, которая говорит привет."""
    print("Привет!")

print(say_hello.__name__)  # Вывод: say_hello
print(say_hello.__doc__)   # Вывод: Функция, которая говорит привет.
```

### Заключение

Декораторы в Python — это мощный и гибкий инструмент, который позволяет изменять поведение функций и методов. Они широко используются в различных библиотеках и фреймворках, таких как Flask и Django, для реализации функциональности, такой как маршрутизация, аутентификация и кэширование. Понимание декораторов поможет вам писать более чистый и поддерживаемый код.

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Что-то происходит перед вызовом функции.")
        func()
        print("Что-то происходит после вызова функции.")
    return wrapper

@my_decorator
def say_hello():
    print("Привет!")

# Вызов функции
say_hello()

def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                func(*args, **kwargs)
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Привет, {name}!")

# Вызов функции
greet("Алиса")

from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Что-то происходит перед вызовом функции.")
        result = func(*args, **kwargs)
        print("Что-то происходит после вызова функции.")
        return result
    return wrapper

@my_decorator
def say_hello():
    """Функция, которая говорит привет."""
    print("Привет!")

print(say_hello.__name__)  # Вывод: say_hello
print(say_hello.__doc__)   # Вывод: Функция, которая говорит привет.

# Extra 
# 1. Ассинхронное программирование

Асинхронное программирование — это парадигма, которая позволяет выполнять операции, не блокируя основной поток выполнения программы. Это особенно полезно в ситуациях, когда необходимо выполнять длительные операции, такие как ввод-вывод (I/O), сетевые запросы или работа с базами данных, не задерживая выполнение других задач.

### Основные концепции асинхронного программирования

1. **Асинхронные функции**: Функции, которые могут приостанавливать своё выполнение и возвращать управление другим функциям. В Python они определяются с помощью ключевого слова `async`.

2. **Ожидание (await)**: Ключевое слово `await` используется для приостановки выполнения асинхронной функции до тех пор, пока не завершится выполнение другой асинхронной функции.

3. **Сопроцессы (coroutines)**: Это функции, которые могут быть приостановлены и возобновлены. В Python сопроцессы создаются с помощью `async def`.

4. **Цикл событий (event loop)**: Это механизм, который управляет выполнением асинхронных задач. Он обрабатывает события и запускает сопроцессы.

### Пример асинхронного программирования

Вот простой пример, который демонстрирует использование асинхронного программирования в Python с помощью модуля `asyncio`:

```python
import asyncio

async def fetch_data():
    print("Начинаем загрузку данных...")
    await asyncio.sleep(2)  # Симуляция длительной операции
    print("Данные загружены!")
    return {"data": 42}

async def main():
    print("Запуск основной функции...")
    data = await fetch_data()  # Ожидание завершения fetch_data
    print(f"Полученные данные: {data}")

# Запуск асинхронной программы
asyncio.run(main())
```

### Объяснение кода

1. **Определение асинхронной функции**: Функция `fetch_data` определена как асинхронная с помощью `async def`. Она использует `await` для приостановки выполнения на 2 секунды, имитируя длительную операцию.

2. **Основная функция**: Функция `main` также является асинхронной и вызывает `fetch_data`, ожидая её завершения.

3. **Запуск программы**: `asyncio.run(main())` запускает цикл событий и выполняет основную функцию.

### Преимущества асинхронного программирования

- **Не блокирует выполнение**: Асинхронные операции позволяют выполнять другие задачи, пока ожидается завершение длительных операций.
- **Эффективное использование ресурсов**: Асинхронное программирование позволяет более эффективно использовать ресурсы, особенно в приложениях, которые требуют высокой производительности и отзывчивости.
- **Упрощение кода**: Асинхронные функции могут сделать код более читаемым и понятным по сравнению с использованием потоков или процессов.

### Заключение

Асинхронное программирование в Python предоставляет мощные инструменты для работы с длительными операциями, позволяя создавать более отзывчивые и эффективные приложения. Использование `async` и `await` в сочетании с модулем `asyncio` позволяет легко управлять асинхронными задачами и упрощает написание кода, который работает с вводом-выводом и сетевыми запросами.

In [ ]:
import asyncio

async def fetch_data():
    print("Начинаем загрузку данных...")
    await asyncio.sleep(2)  # Симуляция длительной операции
    print("Данные загружены!")
    return {"data": 42}

async def main():
    print("Запуск основной функции...")
    data = await fetch_data()  # Ожидание завершения fetch_data
    print(f"Полученные данные: {data}")

# Запуск асинхронной программы
asyncio.run(main())

# 2. Многопоточное программирование, библиотека threading
Многопоточное программирование — это парадигма, позволяющая выполнять несколько потоков (или "нитей") одновременно в рамках одного процесса. Это может быть полезно для выполнения параллельных задач, таких как обработка данных, выполнение сетевых запросов или выполнение длительных операций, не блокируя основной поток выполнения программы.

### Библиотека `threading`

В Python для работы с потоками используется стандартная библиотека `threading`. Она предоставляет высокоуровневый интерфейс для создания и управления потоками.

### Основные компоненты библиотеки `threading`

1. **Thread**: Класс, представляющий поток. Вы можете создать новый поток, передав целевую функцию и её аргументы.
2. **Lock**: Объект, который используется для синхронизации потоков, предотвращая одновременный доступ к общим ресурсам.
3. **Event**: Объект, который позволяет потокам сигнализировать друг другу о событиях.
4. **Condition**: Объект, который позволяет потокам ожидать определённых условий.

### Пример использования библиотеки `threading`

Вот простой пример, который демонстрирует создание и запуск потоков с использованием библиотеки `threading`:

```python
import threading
import time

def worker(thread_id):
    print(f"Поток {thread_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Поток {thread_id} завершил работу.")

# Создание списка потоков
threads = []
for i in range(5):
    thread = threading.Thread(target=worker, args=(i,))
    threads.append(thread)
    thread.start()  # Запуск потока

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print("Все потоки завершили работу.")
```

### Объяснение кода

1. **Импортирование библиотеки**: Мы импортируем библиотеку `threading` и `time` для использования функции `sleep`.

2. **Определение функции `worker`**: Эта функция принимает идентификатор потока и выполняет некоторую работу (в данном случае просто ждет 2 секунды).

3. **Создание и запуск потоков**:
   - Мы создаем список `threads` для хранения потоков.
   - В цикле создаем 5 потоков, передавая функции `worker` идентификатор потока в качестве аргумента.
   - Каждый поток запускается с помощью метода `start()`.

4. **Ожидание завершения потоков**: Мы используем метод `join()`, чтобы дождаться завершения всех потоков перед тем, как продолжить выполнение основной программы.

### Синхронизация потоков

При работе с многопоточностью важно учитывать, что несколько потоков могут одновременно пытаться получить доступ к общим ресурсам, что может привести к состояниям гонки. Для предотвращения этого можно использовать объекты `Lock`.

Вот пример использования `Lock` для синхронизации потоков:

```python
import threading

# Общий ресурс
counter = 0
lock = threading.Lock()

def increment():
    global counter
    for _ in range(100000):
        with lock:  # Захват блокировки
            counter += 1  # Изменение общего ресурса

# Создание потоков
threads = []
for _ in range(2):
    thread = threading.Thread(target=increment)
    threads.append(thread)
    thread.start()

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print(f"Итоговое значение счетчика: {counter}")
```

### Объяснение кода

1. **Общий ресурс**: Мы определяем переменную `counter`, к которой будут обращаться потоки, и создаем объект `Lock`.

2. **Функция `increment`**: Эта функция увеличивает значение `counter` 100000 раз. Блокировка (`lock`) используется для обеспечения того, что только один поток может изменять `counter` в любой момент времени.

3. **Создание и запуск потоков**: Мы создаем два потока, которые выполняют функцию `increment`.

4. **Ожидание завершения потоков**: Мы используем `join()`, чтобы дождаться завершения всех потоков.

### Заключение

Многопоточное программирование в Python с использованием библиотеки `threading` позволяет эффективно выполнять параллельные задачи. Однако важно учитывать проблемы синхронизации и состояния гонки, которые могут возникнуть при работе с общими ресурсами. Использование объектов `Lock` и других механизмов синхронизации помогает избежать этих проблем и обеспечивает корректное выполнение многопоточных программ.

In [ ]:
import threading
import time

def worker(thread_id):
    print(f"Поток {thread_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Поток {thread_id} завершил работу.")

# Создание списка потоков
threads = []
for i in range(5):
    thread = threading.Thread(target=worker, args=(i,))
    threads.append(thread)
    thread.start()  # Запуск потока

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print("Все потоки завершили работу.")

import threading

# Общий ресурс
counter = 0
lock = threading.Lock()

def increment():
    global counter
    for _ in range(100000):
        with lock:  # Захват блокировки
            counter += 1  # Изменение общего ресурса

# Создание потоков
threads = []
for _ in range(2):
    thread = threading.Thread(target=increment)
    threads.append(thread)
    thread.start()

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print(f"Итоговое значение счетчика: {counter}")


# 3. Параллельное программирование, библиотека multiprocessing
Параллельное программирование — это парадигма, позволяющая выполнять несколько процессов одновременно, что может значительно ускорить выполнение задач, особенно на многоядерных процессорах. В Python для реализации параллельного программирования используется библиотека `multiprocessing`, которая позволяет создавать отдельные процессы, каждый из которых имеет собственное пространство памяти.

### Основные компоненты библиотеки `multiprocessing`

1. **Process**: Класс, представляющий отдельный процесс. Вы можете создать новый процесс, передав целевую функцию и её аргументы.
2. **Queue**: Объект, который позволяет обмениваться данными между процессами.
3. **Pipe**: Объект, который позволяет устанавливать двустороннюю связь между двумя процессами.
4. **Pool**: Позволяет создавать пул процессов для выполнения задач параллельно, что упрощает распределение работы.

### Пример использования библиотеки `multiprocessing`

Вот простой пример, который демонстрирует создание и запуск процессов с использованием библиотеки `multiprocessing`:

```python
import multiprocessing
import time

def worker(process_id):
    print(f"Процесс {process_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Процесс {process_id} завершил работу.")

if __name__ == "__main__":
    # Создание списка процессов
    processes = []
    for i in range(5):
        process = multiprocessing.Process(target=worker, args=(i,))
        processes.append(process)
        process.start()  # Запуск процесса

    # Ожидание завершения всех процессов
    for process in processes:
        process.join()

    print("Все процессы завершили работу.")
```

### Объяснение кода

1. **Импортирование библиотеки**: Мы импортируем библиотеку `multiprocessing` и `time` для использования функции `sleep`.

2. **Определение функции `worker`**: Эта функция принимает идентификатор процесса и выполняет некоторую работу (в данном случае просто ждет 2 секунды).

3. **Создание и запуск процессов**:
   - Мы создаем список `processes` для хранения процессов.
   - В цикле создаем 5 процессов, передавая функции `worker` идентификатор процесса в качестве аргумента.
   - Каждый процесс запускается с помощью метода `start()`.

4. **Ожидание завершения процессов**: Мы используем метод `join()`, чтобы дождаться завершения всех процессов перед тем, как продолжить выполнение основной программы.

### Использование `Pool` для параллельного выполнения задач

Библиотека `multiprocessing` также предоставляет класс `Pool`, который позволяет создавать пул процессов и распределять задачи между ними. Это упрощает выполнение большого количества однотипных задач.

Вот пример использования `Pool`:

```python
import multiprocessing
import time

def square(n):
    time.sleep(1)  # Симуляция длительной работы
    return n * n

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]
    
    # Создание пула процессов
    with multiprocessing.Pool(processes=3) as pool:
        results = pool.map(square, numbers)  # Параллельное выполнение функции square

    print("Результаты:", results)
```

### Объяснение кода

1. **Определение функции `square`**: Эта функция принимает число и возвращает его квадрат, имитируя длительную работу с помощью `sleep`.

2. **Создание пула процессов**: Мы создаем пул из 3 процессов с помощью `multiprocessing.Pool`.

3. **Параллельное выполнение**: Метод `map` используется для параллельного выполнения функции `square` для каждого элемента в списке `numbers`. Результаты собираются в список `results`.

4. **Вывод результатов**: После завершения всех процессов выводятся результаты.

### Заключение

Параллельное программирование с использованием библиотеки `multiprocessing` в Python позволяет эффективно использовать многоядерные процессоры для выполнения задач. Это особенно полезно для CPU-bound задач, таких как вычисления, где время выполнения может быть значительно сокращено за счет параллельного выполнения. Библиотека `multiprocessing` предоставляет удобные инструменты для создания процессов, обмена данными и управления выполнением задач.

In [ ]:
import multiprocessing
import time

def worker(process_id):
    print(f"Процесс {process_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Процесс {process_id} завершил работу.")

if __name__ == "__main__":
    # Создание списка процессов
    processes = []
    for i in range(5):
        process = multiprocessing.Process(target=worker, args=(i,))
        processes.append(process)
        process.start()  # Запуск процесса

    # Ожидание завершения всех процессов
    for process in processes:
        process.join()

    print("Все процессы завершили работу.")

import multiprocessing
import time

def square(n):
    time.sleep(1)  # Симуляция длительной работы
    return n * n

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]
    
    # Создание пула процессов
    with multiprocessing.Pool(processes=3) as pool:
        results = pool.map(square, numbers)  # Параллельное выполнение функции square

    print("Результаты:", results)
